# algoritmo_deidentificacion_regex

Este script permite deidentificar texto libre proveniente de registros clínicos electrónicos.

0. Configura el ambiente.
1. Carga los datos (input)
~~Chequea la sanidad de los datos (backlog)~~
2. Preprocesa los textos ("limpieza")
~~Parametriza el modelo de deidentificación a aplicar (cambaa, regex, regex+spacy)~~
~~Chequea la cantidad de textos y de ser necesario, particiona la tarea.~~
3. Parametriza el output deseado (texto censurado, resaltado, reemplazado, clasificado). 
4. Procesa los textos (aplica las reglas).
5. Devuelve los datos procesados (output)


## Lista de entidades

- PACIENTE: PERSONA
- EDAD: 
- GÉNERO: no se detecta. 
- FAMILIAR: PERSONA
- ~~DRX~~
- ~~FECHA~~
- ~~EFECTOR~~
- ~~INSTITUCIÓN~~: escolares.
- ~~DIRECCIÓN~~
- ~~ZONA~~
- ~~PAÍS~~
- ~~NÚM_TELÉFONO~~
- ~~CORREO_ELECTRÓNICO~~
- ~~NÚM_DNI~~
- ~~NÚM_CUIT_CUIL~~
- ~~PASAPORTE~~
- ~~MATRICULA~~
- ~~EPOF:~~ queda pendiente estrategia para reemplazar por el nombre de la EPof entre <>
- ~~PATENTE~~: no se cuenta con ejemplos (no se detectaron en la base de datos).
- NÚM_SERIE_DISPOSITIVOS: no se detectan.
- ~~OTROS_NÚM~~: no se detectan.
- DUDOSOS: no se detectan.

## 0. Configura el ambiente

In [64]:
import pandas as pd
import os
import re

# This code allows to view full lenght text
pd.set_option("display.max_colwidth", None)

In [65]:
def remove_accents(text: str)  -> str:
    """Removes common accent characters, converts to lowercase."""
    new = re.sub(r'[àáâãäå]', 'a', text)
    new = re.sub(r'[èéêë]', 'e', new)
    new = re.sub(r'[ìíîï]', 'i', new)
    new = re.sub(r'[òóôõö]', 'o', new)
    new = re.sub(r'[ùúûü]', 'u', new)
    new = re.sub(r'[ÀÁÂÃÄÅ]', 'A', new)
    new = re.sub(r'[ÈÉÊË]', 'E', new)
    new = re.sub(r'[ÌÍÎÏ]', 'I', new)
    new = re.sub(r'[ÒÓÔÕÖ]', 'O', new)
    new = re.sub(r'[ÙÚÛÜ]', 'U', new)
    return new

## 1. Carga los datos (input)

In [66]:
INPUT_DIR = '/home/usuario/Documentos/TrabajoEspecial/Datasets/SPG/integral'
TEXT_COLUMN = 'informes'

In [67]:
# Obtener lista de archivos .txt
txt_files = sorted([f for f in os.listdir(INPUT_DIR) if f.endswith(".txt")])

# Mostrar los archivos encontrados
print(txt_files[:5])

data = []

for txt_file in txt_files:
    txt_path = os.path.join(INPUT_DIR, txt_file)
    
    with open(txt_path, 'r', encoding='utf-8') as f:
        texto = f.read()
    
    data.append({
        TEXT_COLUMN: texto
    })

documents = pd.DataFrame(data)

# Mostrar primeras filas
print(documents.head(2))

['000096468.txt', '001956719.txt', '002769099.txt', '003666606.txt', '004954246.txt']
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

## 2. Preprocesa los textos ("limpieza")

In [68]:
# Genera una nueva columna con el texto preprocesado
documents.rename(columns={TEXT_COLUMN: 'original'}, inplace=True)
documents['preprocessed'] = documents['original'].apply(remove_accents)
# Apply further transformations if necessary

# Remove nan
documents = documents.dropna()
documents.head(2)

,original,preprocessed
0,"Datos del paciente.\nNombre: María Soledad Moreno Roca\nDNI: 23556552K\nFecha de nacimiento: 09/01/1941\nGénero: Mujer\nDomicilio: Calle de Almagro 80\nCiudad: Denia, Valencia, Comunidad Valenciana\nCódigo postal: 46571\nEmail: mariasoledad_roca@ucm.es\nTeléfono fijo: +34 960 66 89 48\nTeléfono móvil: +34 660 57 14 97\nNHC: 2409425\nNASS: 468043486571\nCondición de riesgo: Científico de Investigación\n\nDatos asistenciales.\nMédico: Dr. Juan Ramón Benito Vicente. NC 097900390. Investigador Clínico en Epidemiología. Instituto de Investigación Biomédica en Red de Enfermedades Infecciosas (CIBERINFEC). Avenida Monforte de Lemos 3-5. 28029. Madrid. España.\nFecha de ingreso: 05/06/1996\nCentro de salud: Centro de Salud Carabanchel\n\nInforme clínico del paciente:\nPaciente sobreviviente de violencia de 55 años de edad, acompañado de su madre.","Datos del paciente.\nNombre: Maria Soledad Moreno Roca\nDNI: 23556552K\nFecha de nacimiento: 09/01/1941\nGenero: Mujer\nDomicilio: Calle de Almagro 80\nCiudad: Denia, Valencia, Comunidad Valenciana\nCodigo postal: 46571\nEmail: mariasoledad_roca@ucm.es\nTelefono fijo: +34 960 66 89 48\nTelefono movil: +34 660 57 14 97\nNHC: 2409425\nNASS: 468043486571\nCondicion de riesgo: Cientifico de Investigacion\n\nDatos asistenciales.\nMedico: Dr. Juan Ramon Benito Vicente. NC 097900390. Investigador Clinico en Epidemiologia. Instituto de Investigacion Biomedica en Red de Enfermedades Infecciosas (CIBERINFEC). Avenida Monforte de Lemos 3-5. 28029. Madrid. España.\nFecha de ingreso: 05/06/1996\nCentro de salud: Centro de Salud Carabanchel\n\nInforme clinico del paciente:\nPaciente sobreviviente de violencia de 55 años de edad, acompañado de su madre."
1,"Datos del paciente.\nNombre: Victoria Plaza Roca\nDNI: 75398682J\nFecha de nacimiento: 27/09/2014\nGénero: F\nDomicilio: Calle de Arganzuela 18\nCiudad: Gijón, La Rioja, La Rioja\nCódigo postal: 26167\nEmail: victoria_roca@ciberer.es\nTeléfono fijo: +34 941 29 92 15\nTeléfono móvil: +34 641 97 74 21\nNHC: 1980495\nNASS: 620412206059\nCondición de riesgo: Operador de Maquinaria Pesada\n\nDatos asistenciales.\nMédico: Dr. Eric Alarcon Pascual. NC 646335336. Neurólogo Investigador en Trastornos Cognitivos. Instituto de Investigación Sanitaria del Hospital 12 de Octubre (Imas12). Avenida de Córdoba, s/n. 28041. Madrid. España.\nFecha de ingreso: 21/10/2021\nEpisodio: 24822158\nCentro de salud: Centro de Salud Málaga Oeste\nMatrícula del coche: 5267NPH\nModelo: Mercedes-Benz C-Class\nVIN: VSL4HQZ1ZGI361218\n\nInforme clínico del paciente:\nPaciente de ascendencia australiana de 7 años de edad, acompañado de su tía. Se registró una sesión de escaneo del iris, el código obtenido para revisarla es KJ369UZN325.","Datos del paciente.\nNombre: Victoria Plaza Roca\nDNI: 75398682J\nFecha de nacimiento: 27/09/2014\nGenero: F\nDomicilio: Calle de Arganzuela 18\nCiudad: Gijon, La Rioja, La Rioja\nCodigo postal: 26167\nEmail: victoria_roca@ciberer.es\nTelefono fijo: +34 941 29 92 15\nTelefono movil: +34 641 97 74 21\nNHC: 1980495\nNASS: 620412206059\nCondicion de riesgo: Operador de Maquinaria Pesada\n\nDatos asistenciales.\nMedico: Dr. Eric Alarcon Pascual. NC 646335336. Neurologo Investigador en Trastornos Cognitivos. Instituto de Investigacion Sanitaria del Hospital 12 de Octubre (Imas12). Avenida de Cordoba, s/n. 28041. Madrid. España.\nFecha de ingreso: 21/10/2021\nEpisodio: 24822158\nCentro de salud: Centro de Salud Malaga Oeste\nMatricula del coche: 5267NPH\nModelo: Mercedes-Benz C-Class\nVIN: VSL4HQZ1ZGI361218\n\nInforme clinico del paciente:\nPaciente de ascendencia australiana de 7 años de edad, acompañado de su tia. Se registro una sesion de escaneo del iris, el codigo obtenido para revisarla es KJ369UZN325."


## 3. Parametriza el output deseado 

Ejemplos de outputs posibles:

1. Texto resaltado: La usuaria <María>
2. Texto censurado: La usuaria xxxx
3. Texto clasificado: La usuaria PACIENTE

In [69]:
from enum import Enum

class AnonymizationMode(Enum):
    HIGHLIGHT = 1
    CENSOR = 2
    REPLACE = 3

# Ingresar el tipo de output deseado
OUTPUT_TYPE = AnonymizationMode.REPLACE

ILLEGAL_CHAR_REGEX = r"[\/<>\:\"\\|\?\*\+\[\]\(\)]"

In [70]:
class RegexAnnonimizer(object):
    """
    The RegexAnonymizer class is designed to find and replace matches of a given regular expression
    within a text string, according to a specified output type. The purpose of this class is to 
    anonymize text by either highlighting, censoring, or replacing specific patterns in the text.

    Attributes:
        regex (str): The regular expression pattern used to identify the text to be anonymized.
        output_type (AnonymizationMode): The mode of anonymization, determining the action taken on matches.
        category (str): The text used for replacement when using the REPLACE output type.
        verbose (bool): Whether to print the replaced text. Useful for debug.

    Methods:
        anonymize(text: str) -> str:
            Anonymizes the input text based on the provided regular expression and output type.
            Depending on the output type, it either highlights, censors, or replaces the matched text.

    Output Types:
        1 (AnonymizationMode.HIGHLIGHT): Wraps the matched text with angle brackets.
            Example: "The user Maria" -> "The user <Maria>"
        2 (AnonymizationMode.CENSOR): Replaces the matched text with 'x' characters, maintaining the length of the original text.
            Example: "The user Maria" -> "The user xxxxx"
        3 (AnonymizationMode.REPLACE): Replaces the matched text with the provided replacement text.
            Example: "The user Maria" -> "The user PACIENTE"
    """
    def __init__(self,
                 content_regex: str,
                 output_type: AnonymizationMode = AnonymizationMode.REPLACE,
                 category: str = None,
                 re_flags: re.RegexFlag = None,
                 verbose: bool = False):
        """
        Initializes the RegexAnonymizer instance with the specified parameters.
        
        Args:
            content_regex (str): The regular expression pattern used to identify the text for anonymization.
            output_type (AnonymizationMode, optional): The mode of anonymization, determining the action taken on matches.
                Default is AnonymizationMode.REPLACE.
            category (str, optional): The text used for replacement when using the REPLACE output type. Required
                if output_type is AnonymizationMode.REPLACE.
            re_flags (re.RegexFlag, optional): flags to pass to re.compile when compiling patterns. Default is None.
            verbose (bool, optional): If True, enables verbose output, providing details about the anonymization
                process. Default is False.
        
        Raises:
            ValueError: If output_type is not an instance of AnonymizationMode.
            ValueError: If output_type is AnonymizationMode.REPLACE and category is not provided.
        """
        if not isinstance(output_type, AnonymizationMode):
            raise ValueError('`output_type` must be an instance of `AnonymizationMode`.')
        self.output_type = output_type
        if re_flags is None:
            self.content_regex = re.compile(content_regex)
        else:
            self.content_regex = re.compile(content_regex, re_flags)
        if self.output_type == AnonymizationMode.REPLACE and not category:
            raise ValueError('When `output_type` is `AnonymizationMode.REPLACE`, you need to provide '
                             'a valid string value for `category`.')
        self.category = category
        self.verbose = verbose

    def _replace_match(self, match: re.Match) -> str:
        """Replaces all matches in string according to self.output_type."""
        matched_name = match.group(0)
        if self.verbose:
            # TODO replace with logger
            print('Replacing', matched_name)

        if self.output_type == AnonymizationMode.HIGHLIGHT:
            return f"<{matched_name}>"
        if self.output_type == AnonymizationMode.CENSOR:
            return "x" * len(matched_name)
        if self.output_type == AnonymizationMode.REPLACE:
            return self.category
        raise ValueError('Output type not suported', self.output_type)

    def _post_process(self, anonymized_text: str) -> str:
        """Collapse consecutive replacements into a single occurrence of category."""
        if self.output_type == AnonymizationMode.REPLACE:
            anonymized_text = re.sub(r'({})(\s+\1)+'.format(self.category), r'\1', anonymized_text)
        return anonymized_text

    def anonymize(self, text: str) -> str:
        """Anonymizes text by replacing content matches, according to output type."""
        # Replace the matched names with the desired output
        anonymized_text = self.content_regex.sub(self._replace_match, text)
        return self._post_process(anonymized_text)

class PrefixRegexAnnonimizer(RegexAnnonimizer):
    """
    A specialized anonymizer that detects and replaces text based on a prefix and content pattern.
    
    This class inherits from RegexAnonymizer and extends its functionality by allowing two regular
    expressions to be specified: one for the prefix and another for the content. If the content is 
    found immediately following the prefix, it is subject to anonymization according to the specified 
    output type. This is particularly useful for identifying and anonymizing structured information, 
    
    such as phone numbers, addresses, or any other context-specific data that is preceded by a 
    particular keyword or phrase.
    
    Attributes:
        prefix_regex (str): A regular expression pattern to identify the prefix text.
        content_regex (str): A regular expression pattern to identify the content to be anonymized.
        output_type (AnonymizationMode): Specifies how the matched content should be anonymized.
        category (str): The replacement text used when the output_type is AnonymizationMode.REPLACE.
        verbose (bool): Determines if detailed information about the anonymization process should be printed.
    
    Methods:
        anonymize(text: str) -> str:
            Processes the input text and replaces occurrences of content following the specified 
            prefix according to the anonymization rules defined by the output_type.
    """
    def __init__(self,
                 prefix_regex: str,
                 content_regex: str,
                 output_type: AnonymizationMode = AnonymizationMode.REPLACE,
                 category: str = None,
                 re_flags: re.RegexFlag = None,
                 verbose: bool = False):
        """
        Initializes the PrefixRegexAnnonimizer instance with the specified parameters.
        
        Args:
            prefix_regex: (str): The regular expression pattern used to identify the prefix before the text to anonymize.
            content_regex: (str): The regular expression pattern used to identify the text for anonymization.
            output_type (AnonymizationMode, optional): The mode of anonymization, determining the action taken on matches.
                Default is AnonymizationMode.REPLACE.
            category (str, optional): The text used for replacement when using the REPLACE output type. Required
                if output_type is AnonymizationMode.REPLACE.
            re_flags (re.RegexFlag, optional): flags to pass to re.compile when compiling patterns. Default is None.
            verbose (bool, optional): If True, enables verbose output, providing details about the anonymization
                process. Default is False.
        
        Raises:
            ValueError: If output_type is not an instance of AnonymizationMode.
            ValueError: If output_type is AnonymizationMode.REPLACE and category is not provided.
        """
        super().__init__(
            content_regex=content_regex,
            output_type=output_type,
            category=category,
            verbose=verbose,
            re_flags=re_flags
        )
        if re_flags is None:
            self.prefix_regex = re.compile(prefix_regex)
            self._full_regex = re.compile(fr'(?P<prefix>{prefix_regex})\s*(?P<content>{content_regex})')
        else:
            self.prefix_regex = re.compile(prefix_regex, re_flags)
            self._full_regex = re.compile(fr'(?P<prefix>{prefix_regex})\s*(?P<content>{content_regex})', re_flags)
    
    def _replace_if_prefix(self, match: re.Match):
        """Replace matches to self. of regex only if matches self._full_regex.""" 
        # Check if a phone-related term precedes the number, and then replace
        return match.group('prefix') + ' ' + self.content_regex.sub(self._replace_match, match.group('content'), count=1)

    def anonymize(self, text: str) -> str:
        """Anonymizes text by replacing content matches that follow prefix matches, according to output type."""
        # Replace the matched names with the desired output
        anonymized_text = self._full_regex.sub(self._replace_if_prefix, text)
        return self._post_process(anonymized_text)

## 4. Procesa los textos (aplica las reglas).

### FECHA

In [71]:
import datetime
## Expresiones regulares para FECHA
regexp_day = r'(\d{1,2}[\/ |-](\d{1,2}|Enero|Ene|Febrero|Feb|Marzo|Mar|Abril|Abr|Mayo|May|Junio|Jun|Julio|Jul|Agosto|Ago|Septiembre|Sep|Octubre|Oct|Noviembre|Nov|Diciembre|Dic)[\/ |-]\d{2,4})'
regexp_month = r'\b(Enero|Ene|Febrero|Feb|Marzo|Mar|Abril|Abr|Mayo|May|Junio|Jun|Julio|Jul|Agosto|Ago|Septiembre|Sep|Octubre|Oct|Noviembre|Nov|Diciembre|Dic)\b'
regexp_year = r'\b(19[89][0-9]|20[0-4][0-9]|{})\b'.format(datetime.date.today().year)
regexp_date = '|'.join([regexp_day, regexp_month, regexp_year])

date_anonymizer = RegexAnnonimizer(
    content_regex=regexp_date,
    category='XFECHAX',
    re_flags=re.IGNORECASE,
    output_type=OUTPUT_TYPE
)
# Ejemplos:
print(date_anonymizer.anonymize("Se programa turno para dìa 21/03/2021. Jueves 17:00hrs, 31 de Marzo."))
print(date_anonymizer.anonymize("Se programa turno para octubre. Control 2023"))

Se programa turno para dìa XFECHAX. Jueves 17:00hrs, 31 de XFECHAX.
Se programa turno para XFECHAX. Control XFECHAX


In [72]:
documents['anonymized'] = documents['preprocessed'].apply(date_anonymizer.anonymize)

In [73]:
documents[documents['anonymized'].str.contains('XFECHAX', case=False)].head(2)

,original,preprocessed,anonymized
0,"Datos del paciente.\nNombre: María Soledad Moreno Roca\nDNI: 23556552K\nFecha de nacimiento: 09/01/1941\nGénero: Mujer\nDomicilio: Calle de Almagro 80\nCiudad: Denia, Valencia, Comunidad Valenciana\nCódigo postal: 46571\nEmail: mariasoledad_roca@ucm.es\nTeléfono fijo: +34 960 66 89 48\nTeléfono móvil: +34 660 57 14 97\nNHC: 2409425\nNASS: 468043486571\nCondición de riesgo: Científico de Investigación\n\nDatos asistenciales.\nMédico: Dr. Juan Ramón Benito Vicente. NC 097900390. Investigador Clínico en Epidemiología. Instituto de Investigación Biomédica en Red de Enfermedades Infecciosas (CIBERINFEC). Avenida Monforte de Lemos 3-5. 28029. Madrid. España.\nFecha de ingreso: 05/06/1996\nCentro de salud: Centro de Salud Carabanchel\n\nInforme clínico del paciente:\nPaciente sobreviviente de violencia de 55 años de edad, acompañado de su madre.","Datos del paciente.\nNombre: Maria Soledad Moreno Roca\nDNI: 23556552K\nFecha de nacimiento: 09/01/1941\nGenero: Mujer\nDomicilio: Calle de Almagro 80\nCiudad: Denia, Valencia, Comunidad Valenciana\nCodigo postal: 46571\nEmail: mariasoledad_roca@ucm.es\nTelefono fijo: +34 960 66 89 48\nTelefono movil: +34 660 57 14 97\nNHC: 2409425\nNASS: 468043486571\nCondicion de riesgo: Cientifico de Investigacion\n\nDatos asistenciales.\nMedico: Dr. Juan Ramon Benito Vicente. NC 097900390. Investigador Clinico en Epidemiologia. Instituto de Investigacion Biomedica en Red de Enfermedades Infecciosas (CIBERINFEC). Avenida Monforte de Lemos 3-5. 28029. Madrid. España.\nFecha de ingreso: 05/06/1996\nCentro de salud: Centro de Salud Carabanchel\n\nInforme clinico del paciente:\nPaciente sobreviviente de violencia de 55 años de edad, acompañado de su madre.","Datos del paciente.\nNombre: Maria Soledad Moreno Roca\nDNI: 23556552K\nFecha de nacimiento: XFECHAX\nGenero: Mujer\nDomicilio: Calle de Almagro 80\nCiudad: Denia, Valencia, Comunidad Valenciana\nCodigo postal: 46571\nEmail: mariasoledad_roca@ucm.es\nTelefono fijo: +34 9XFECHAX 48\nTelefono movil: +34 6XFECHAX 97\nNHC: 2409425\nNASS: 468043486571\nCondicion de riesgo: Cientifico de Investigacion\n\nDatos asistenciales.\nMedico: Dr. Juan Ramon Benito Vicente. NC 097900390. Investigador Clinico en Epidemiologia. Instituto de Investigacion Biomedica en Red de Enfermedades Infecciosas (CIBERINFEC). Avenida Monforte de Lemos 3-5. 28029. Madrid. España.\nFecha de ingreso: XFECHAX\nCentro de salud: Centro de Salud Carabanchel\n\nInforme clinico del paciente:\nPaciente sobreviviente de violencia de 55 años de edad, acompañado de su madre."
1,"Datos del paciente.\nNombre: Victoria Plaza Roca\nDNI: 75398682J\nFecha de nacimiento: 27/09/2014\nGénero: F\nDomicilio: Calle de Arganzuela 18\nCiudad: Gijón, La Rioja, La Rioja\nCódigo postal: 26167\nEmail: victoria_roca@ciberer.es\nTeléfono fijo: +34 941 29 92 15\nTeléfono móvil: +34 641 97 74 21\nNHC: 1980495\nNASS: 620412206059\nCondición de riesgo: Operador de Maquinaria Pesada\n\nDatos asistenciales.\nMédico: Dr. Eric Alarcon Pascual. NC 646335336. Neurólogo Investigador en Trastornos Cognitivos. Instituto de Investigación Sanitaria del Hospital 12 de Octubre (Imas12). Avenida de Córdoba, s/n. 28041. Madrid. España.\nFecha de ingreso: 21/10/2021\nEpisodio: 24822158\nCentro de salud: Centro de Salud Málaga Oeste\nMatrícula del coche: 5267NPH\nModelo: Mercedes-Benz C-Class\nVIN: VSL4HQZ1ZGI361218\n\nInforme clínico del paciente:\nPaciente de ascendencia australiana de 7 años de edad, acompañado de su tía. Se registró una sesión de escaneo del iris, el código obtenido para revisarla es KJ369UZN325.","Datos del paciente.\nNombre: Victoria Plaza Roca\nDNI: 75398682J\nFecha de nacimiento: 27/09/2014\nGenero: F\nDomicilio: Calle de Arganzuela 18\nCiudad: Gijon, La Rioja, La Rioja\nCodigo postal: 26167\nEmail: victoria_roca@ciberer.es\nTelefono fijo: +34 941 29 92 15\nTelefono movil: +34 641 97 74 21\nNHC: 1980495\nNASS: 620412206059\nCondicion de riesgo: Operador de Maquinaria Pesada\n\n

### NUM_TELEFONO

In [74]:
## Expresión regular para NUM_TELEFONO
regexp_phone_prefix =  '(tel[ée]fono|(tel(?![aA-zZ]))|celular|(cel(?![aA-zZ])))\s*(?:n\s*[\.\°]|.)?\s*[:\.]?'
regexp_phone_number = '[0-9]+'
separators = r'\s*[:\.]?\s*'

phone_anonymizer = PrefixRegexAnnonimizer(
    prefix_regex=regexp_phone_prefix,
    content_regex=regexp_phone_number,
    re_flags=re.IGNORECASE,
    category='XTELEFONOX',
    output_type=OUTPUT_TYPE,
)

print(phone_anonymizer.anonymize("Paciente indica llamar a tel 3514790943 sin poder comunicarse."))
print(phone_anonymizer.anonymize("Paciente indica llamar a telefono n° 3514790943 sin poder comunicarse."))
print(phone_anonymizer.anonymize("Rescinde celular durante 400 dias"))
print(phone_anonymizer.anonymize("Proximo turno enero, mandar recordatorio cel: 3514790943"))
print(phone_anonymizer.anonymize("Proximo turno enero, mandar recordatorio cel.: 3514790943"))

Paciente indica llamar a tel 3 XTELEFONOX sin poder comunicarse.
Paciente indica llamar a telefono n°  XTELEFONOX sin poder comunicarse.
Rescinde celular durante 400 dias
Proximo turno enero, mandar recordatorio cel:  XTELEFONOX
Proximo turno enero, mandar recordatorio cel.: XTELEFONOX


<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_63444/1487379825.py:2: SyntaxWarning: invalid escape sequence '\s'
  regexp_phone_prefix =  '(tel[ée]fono|(tel(?![aA-zZ]))|celular|(cel(?![aA-zZ])))\s*(?:n\s*[\.\°]|.)?\s*[:\.]?'


In [75]:
documents['anonymized'] = documents.anonymized.apply(phone_anonymizer.anonymize)

In [76]:
documents[documents['anonymized'].str.contains('XNUM_TELEFONOX',case=False)].head(2)

,original,preprocessed,anonymized


### CORREO ELECTRÓNICO

In [77]:
## Expresión regular para CORREO ELECTRÓNICO
regexp_email = r'[\w\.-]+@[\w\.-]+\.\w+'

email_anonymizer = RegexAnnonimizer(
    content_regex=regexp_email,
    category='XCORREO_ELECTRONICOX',
    re_flags=re.IGNORECASE,
    output_type=OUTPUT_TYPE,
)

print(email_anonymizer.anonymize("Contactar paciente a fakemail@fakedomain.com"))
print(email_anonymizer.anonymize("Contactar paciente a fakemail@fakedomain.com.ar"))
print(email_anonymizer.anonymize("Paciente refiere conflicto con herman@"))

Contactar paciente a XCORREO_ELECTRONICOX
Contactar paciente a XCORREO_ELECTRONICOX
Paciente refiere conflicto con herman@


In [78]:
documents['anonymized'] = documents['anonymized'].apply(email_anonymizer.anonymize)

In [79]:
documents[documents['anonymized'].str.contains('XCORREO_ELECTRONICOX',case=False)]

,original,preprocessed,anonymized
0,"Datos del paciente.\nNombre: María Soledad Moreno Roca\nDNI: 23556552K\nFecha de nacimiento: 09/01/1941\nGénero: Mujer\nDomicilio: Calle de Almagro 80\nCiudad: Denia, Valencia, Comunidad Valenciana\nCódigo postal: 46571\nEmail: mariasoledad_roca@ucm.es\nTeléfono fijo: +34 960 66 89 48\nTeléfono móvil: +34 660 57 14 97\nNHC: 2409425\nNASS: 468043486571\nCondición de riesgo: Científico de Investigación\n\nDatos asistenciales.\nMédico: Dr. Juan Ramón Benito Vicente. NC 097900390. Investigador Clínico en Epidemiología. Instituto de Investigación Biomédica en Red de Enfermedades Infecciosas (CIBERINFEC). Avenida Monforte de Lemos 3-5. 28029. Madrid. España.\nFecha de ingreso: 05/06/1996\nCentro de salud: Centro de Salud Carabanchel\n\nInforme clínico del paciente:\nPaciente sobreviviente de violencia de 55 años de edad, acompañado de su madre.","Datos del paciente.\nNombre: Maria Soledad Moreno Roca\nDNI: 23556552K\nFecha de nacimiento: 09/01/1941\nGenero: Mujer\nDomicilio: Calle de Almagro 80\nCiudad: Denia, Valencia, Comunidad Valenciana\nCodigo postal: 46571\nEmail: mariasoledad_roca@ucm.es\nTelefono fijo: +34 960 66 89 48\nTelefono movil: +34 660 57 14 97\nNHC: 2409425\nNASS: 468043486571\nCondicion de riesgo: Cientifico de Investigacion\n\nDatos asistenciales.\nMedico: Dr. Juan Ramon Benito Vicente. NC 097900390. Investigador Clinico en Epidemiologia. Instituto de Investigacion Biomedica en Red de Enfermedades Infecciosas (CIBERINFEC). Avenida Monforte de Lemos 3-5. 28029. Madrid. España.\nFecha de ingreso: 05/06/1996\nCentro de salud: Centro de Salud Carabanchel\n\nInforme clinico del paciente:\nPaciente sobreviviente de violencia de 55 años de edad, acompañado de su madre.","Datos del paciente.\nNombre: Maria Soledad Moreno Roca\nDNI: 23556552K\nFecha de nacimiento: XFECHAX\nGenero: Mujer\nDomicilio: Calle de Almagro 80\nCiudad: Denia, Valencia, Comunidad Valenciana\nCodigo postal: 46571\nEmail: XCORREO_ELECTRONICOX\nTelefono fijo: +34 9XFECHAX 48\nTelefono movil: +34 6XFECHAX 97\nNHC: 2409425\nNASS: 468043486571\nCondicion de riesgo: Cientifico de Investigacion\n\nDatos asistenciales.\nMedico: Dr. Juan Ramon Benito Vicente. NC 097900390. Investigador Clinico en Epidemiologia. Instituto de Investigacion Biomedica en Red de Enfermedades Infecciosas (CIBERINFEC). Avenida Monforte de Lemos 3-5. 28029. Madrid. España.\nFecha de ingreso: XFECHAX\nCentro de salud: Centro de Salud Carabanchel\n\nInforme clinico del paciente:\nPaciente sobreviviente de violencia de 55 años de edad, acompañado de su madre."
1,"Datos del paciente.\nNombre: Victoria Plaza Roca\nDNI: 75398682J\nFecha de nacimiento: 27/09/2014\nGénero: F\nDomicilio: Calle de Arganzuela 18\nCiudad: Gijón, La Rioja, La Rioja\nCódigo postal: 26167\nEmail: victoria_roca@ciberer.es\nTeléfono fijo: +34 941 29 92 15\nTeléfono móvil: +34 641 97 74 21\nNHC: 1980495\nNASS: 620412206059\nCondición de riesgo: Operador de Maquinaria Pesada\n\nDatos asistenciales.\nMédico: Dr. Eric Alarcon Pascual. NC 646335336. Neurólogo Investigador en Trastornos Cognitivos. Instituto de Investigación Sanitaria del Hospital 12 de Octubre (Imas12). Avenida de Córdoba, s/n. 28041. Madrid. España.\nFecha de ingreso: 21/10/2021\nEpisodio: 24822158\nCentro de salud: Centro de Salud Málaga Oeste\nMatrícula del coche: 5267NPH\nModelo: Mercedes-Benz C-Class\nVIN: VSL4HQZ1ZGI361218\n\nInforme clínico del paciente:\nPaciente de ascendencia australiana de 7 años de edad, acompañado de su tía. Se registró una sesión de escaneo del iris, el código obtenido para revisarla es KJ369UZN325.","Datos del paciente.\nNombre: Victoria Plaza Roca\nDNI: 75398682J\nFecha de nacimiento: 27/09/2014\nGenero: F\nDomicilio: Calle de Arganzuela 18\nCiudad: Gijon, La Rioja, La Rioja\nCodigo postal: 26167\nEmail: victoria_roca@ciberer.es\nTelefono fijo: +34 941 29 92 15\nTelefono movil: +34 641 97 74 21\nNHC: 1980495\nNASS: 620412206059\nCondicion de riesgo: Operador de Maquinaria Pesada\n\nDato

### NUM_DNI

In [80]:
## Expresión regular para NUM_DNI

regexp_dni_prefix = r'(d\.n\.i|dni|doc(?= )|documento)(\sn\s?\.?\°?)?\s*[:\.]?\s*'
regexp_dni_numbers = r'\b\d{1,3}(?:\.*\d{3})*\b'

dni_anonymizer = PrefixRegexAnnonimizer(
    prefix_regex=regexp_dni_prefix,
    content_regex=regexp_dni_numbers,
    re_flags=re.IGNORECASE,
    category='XNUM_DNIX',
    output_type=OUTPUT_TYPE,
)

print(dni_anonymizer.anonymize("Paciente dni 27.384.234 control prenatal semana 27"))
print(dni_anonymizer.anonymize("Paciente dni n° 27.384.234 control prenatal semana 27"))
print(dni_anonymizer.anonymize("Paciente dni n ° 27.384.234 control prenatal semana 27"))
print(dni_anonymizer.anonymize("Paciente dni 27384234, control prenatal"))
print(dni_anonymizer.anonymize("Paciente dni: 27384234 control prenatal"))
print(dni_anonymizer.anonymize("Para mas detalles ver documento 25"))
print(dni_anonymizer.anonymize("Refiere extravio dni sin sintomatologia tel 27384234"))

Paciente dni  XNUM_DNIX control prenatal semana 27
Paciente dni n°  XNUM_DNIX control prenatal semana 27
Paciente dni n °  XNUM_DNIX control prenatal semana 27
Paciente dni  XNUM_DNIX, control prenatal
Paciente dni:  XNUM_DNIX control prenatal
Para mas detalles ver documento  XNUM_DNIX
Refiere extravio dni sin sintomatologia tel 27384234


In [81]:
documents['anonymized'] = documents['anonymized'].apply(dni_anonymizer.anonymize)

In [82]:
documents[documents['anonymized'].str.contains('XNUM_DNIX',case=False)]

,original,preprocessed,anonymized


### NUM_CUIT_CUIL

In [83]:
## Expresión regular para NUM_CUIT_CUIL
regexp_cuil_prefix = r'(c.u.i.l.|cuil|c.u.i.t|cuit)(\sn\s?\.?\°?)?\s*[:\.]?\s*'
# Numbers, or numbers with . or -
regexp_cuil_numbers = r'\b(?:\d(?:[\.\-]\d)*)+\b'

cuit_anonymizer = PrefixRegexAnnonimizer(
    prefix_regex=regexp_cuil_prefix,
    content_regex=regexp_cuil_numbers,
    re_flags=re.IGNORECASE,
    category='XNUM_CUIT_CUILX',
    output_type=OUTPUT_TYPE,
)

print(cuit_anonymizer.anonymize("Paciente CUIT 25-27.384.234-4 control prenatal semana 27"))
print(cuit_anonymizer.anonymize("Paciente cuil 25-27384234-4, control prenatal"))
print(cuit_anonymizer.anonymize("Paciente c.u.i.t 25273842344 control prenatal"))
print(cuit_anonymizer.anonymize("Refiere extravio cuit sin sintomatologia tel 27384234"))

Paciente CUIT  XNUM_CUIT_CUILX control prenatal semana 27
Paciente cuil  XNUM_CUIT_CUILX, control prenatal
Paciente c.u.i.t  XNUM_CUIT_CUILX control prenatal
Refiere extravio cuit sin sintomatologia tel 27384234


In [84]:
documents['anonymized'] = documents['anonymized'].apply(cuit_anonymizer.anonymize)

In [85]:
documents[documents['anonymized'].str.contains('XNUM_CUIT_CUILX',case=False)]

,original,preprocessed,anonymized


### PASAPORTE

In [86]:
## Expresión regular para Pasaporte

regexp_passport_prefix = r'pasaporte(\sn\s?\.?\°?)?\s*[:\.]?\s*'
# Between 3 and 10 alphanumeric characters with at least one number
regexp_passport_numbers = r'\b[a-zA-Z0-9]*[0-9][a-zA-Z0-9]*\b'

passport_anonymizer = PrefixRegexAnnonimizer(
    prefix_regex=regexp_passport_prefix,
    content_regex=regexp_passport_numbers,
    re_flags=re.IGNORECASE,
    category='XPASAPORTEX',
    output_type=OUTPUT_TYPE,
)

print(passport_anonymizer.anonymize("Paciente pasaporte AAA342034, control prenatal semana 27"))
print(passport_anonymizer.anonymize("Paciente pasaporte n AAA342034, control prenatal semana 27"))
print(passport_anonymizer.anonymize("Paciente pasaporte n°: AAA342034, control prenatal semana 27"))
print(passport_anonymizer.anonymize("Paciente pasaporte 2527384234"))
print(passport_anonymizer.anonymize("Refiere extravio pasaporte sin sintomatologia tel 27384234"))

Paciente pasaporte  XPASAPORTEX, control prenatal semana 27
Paciente pasaporte n  XPASAPORTEX, control prenatal semana 27
Paciente pasaporte n°:  XPASAPORTEX, control prenatal semana 27
Paciente pasaporte  XPASAPORTEX
Refiere extravio pasaporte sin sintomatologia tel 27384234


In [87]:
documents['anonymized'] = documents['anonymized'].apply(passport_anonymizer.anonymize)

In [88]:
documents[documents['anonymized'].str.contains('XPASAPORTEX',case=False)]

,original,preprocessed,anonymized


### MATRICULA

In [89]:
## Expresión regular para MATRICULA
regexp_license_prefix = r'\bm(\.?)(p|n)(\.?)\s*[:\.]?\s*'
regexp_license_numbers = r'\b[0-9]{3,}\b'

license_anonymizer = PrefixRegexAnnonimizer(
    prefix_regex=regexp_license_prefix,
    content_regex=regexp_license_numbers,
    re_flags=re.IGNORECASE,
    category='XMATRICULAX',
    output_type=OUTPUT_TYPE,
)

print(license_anonymizer.anonymize("Atendido por dr. Ortiz mp 23411, control prenatal semana 27"))
print(license_anonymizer.anonymize("Atendido por dr. Ortiz m.p. 23411 control prenatal semana 27"))
print(license_anonymizer.anonymize("Atendido por dr. Ortiz m.n.: 23411 control prenatal semana 27"))
print(license_anonymizer.anonymize("Refiere extravio m.p. sin sintomatologia tel 27384234"))

Atendido por dr. Ortiz mp  XMATRICULAX, control prenatal semana 27
Atendido por dr. Ortiz m.p.  XMATRICULAX control prenatal semana 27
Atendido por dr. Ortiz m.n.:  XMATRICULAX control prenatal semana 27
Refiere extravio m.p. sin sintomatologia tel 27384234


In [90]:
documents['anonymized'] = documents['anonymized'].apply(license_anonymizer.anonymize)

In [91]:
documents[documents['anonymized'].str.contains('XMATRICULAX',case=False)]

,original,preprocessed,anonymized


### PATENTE

In [92]:
## Expresión regular para PATENTE
regexp_license_plate = r'\b([aA-zZ]{3}[0-9]{3})|([aA-zZ]{2}[0-9]{3}[aA-zZ]{2})\b'

license_plate_anonymizer = RegexAnnonimizer(
    content_regex=regexp_license_plate,
    category='XPATENTEX',
    re_flags=re.IGNORECASE,
    output_type=OUTPUT_TYPE,
)

print(license_plate_anonymizer.anonymize("Auto patente AA345QP fuera de servicio"))
print(license_plate_anonymizer.anonymize("Auto patente aub325- fuera de servicio"))

Auto patente XPATENTEX fuera de servicio
Auto patente XPATENTEX- fuera de servicio


In [93]:
documents['anonymized'] = documents['anonymized'].apply(license_plate_anonymizer.anonymize)

In [94]:
documents[documents['anonymized'].str.contains('XPATENTEX', case=False)]

,original,preprocessed,anonymized
16,"Datos del paciente.\nNombre: Sonia Giménez Mateo\nDNI: 28458568R\nFecha de nacimiento: 12/10/1979\nGénero: F\nDomicilio: Calle de Amor de Dios 28\nCiudad: Málaga, Región de Murcia, Región de Murcia\nCódigo postal: 30174\nEmail: sonia-mateo@udc.es\nTeléfono fijo: +34 968 81 89 78\nTeléfono móvil: +34 668 04 26 48\nNHC: 3502518\nNASS: 148472849379\n\nDatos asistenciales.\nMédico: Dra. Remedios Torres Gutiérrez. NC 729610890. Investigadora Clínica en Medicina Interna. Instituto Maimónides de Investigación Biomédica de Córdoba (IMIBIC). Avenida Menéndez Pidal, s/n. 14004. Córdoba. España.\nFecha de ingreso: 22/01/1987\nHospital: Hospital Universitario de Basurto\nMatrícula del coche: 4884RKT\nModelo: Toyota RAV4\nVIN: VSU906M506F437481\n\nInforme clínico del paciente:\nPaciente de ascendencia gitana de 7 años de edad, acompañado de su hermana. Se registró una sesión de reconocimiento facial, el código obtenido para revisarla es FZ253TPZ757. Se registró una sesión de escaneo del iris, el código obtenido para revisarla es OS931MNR959.","Datos del paciente.\nNombre: Sonia Gimenez Mateo\nDNI: 28458568R\nFecha de nacimiento: 12/10/1979\nGenero: F\nDomicilio: Calle de Amor de Dios 28\nCiudad: Malaga, Region de Murcia, Region de Murcia\nCodigo postal: 30174\nEmail: sonia-mateo@udc.es\nTelefono fijo: +34 968 81 89 78\nTelefono movil: +34 668 04 26 48\nNHC: 3502518\nNASS: 148472849379\n\nDatos asistenciales.\nMedico: Dra. Remedios Torres Gutierrez. NC 729610890. Investigadora Clinica en Medicina Interna. Instituto Maimonides de Investigacion Biomedica de Cordoba (IMIBIC). Avenida Menendez Pidal, s/n. 14004. Cordoba. España.\nFecha de ingreso: 22/01/1987\nHospital: Hospital Universitario de Basurto\nMatricula del coche: 4884RKT\nModelo: Toyota RAV4\nVIN: VSU906M506F437481\n\nInforme clinico del paciente:\nPaciente de ascendencia gitana de 7 años de edad, acompañado de su hermana. Se registro una sesion de reconocimiento facial, el codigo obtenido para revisarla es FZ253TPZ757. Se registro una sesion de escaneo del iris, el codigo obtenido para revisarla es OS931MNR959.","Datos del paciente.\nNombre: Sonia Gimenez Mateo\nDNI: 28458568R\nFecha de nacimiento: XFECHAX\nGenero: F\nDomicilio: Calle de Amor de Dios 28\nCiudad: Malaga, Region de Murcia, Region de Murcia\nCodigo postal: 30174\nEmail: XCORREO_ELECTRONICOX\nTelefono fijo: +34 9XFECHAX 78\nTelefono movil: +34 6XFECHAX 48\nNHC: 3502518\nNASS: 148472849379\n\nDatos asistenciales.\nMedico: Dra. Remedios Torres Gutierrez. NC 729610890. Investigadora Clinica en Medicina Interna. Instituto Maimonides de Investigacion Biomedica de Cordoba (IMIBIC). Avenida Menendez Pidal, s/n. 14004. Cordoba. España.\nFecha de ingreso: XFECHAX\nHospital: Hospital Universitario de Basurto\nMatricula del coche: 4884RKT\nModelo: Toyota RAV4\nVIN: XPATENTEXM506F437481\n\nInforme clinico del paciente:\nPaciente de ascendencia gitana de 7 años de edad, acompañado de su hermana. Se registro una sesion de reconocimiento facial, el codigo obtenido para revisarla es FZ253TPZ757. Se registro una sesion de escaneo del iris, el codigo obtenido para revisarla es OS931MNR959."
33,"Datos del paciente.\nNombre: Borja Silva Mateos\nDNI: 60347220G\nFecha de nacimiento: 11/03/2005\nGénero: H\nDomicilio: Calle de Ferraz 63\nCiudad: Santa Cruz de Tenerife, Teruel, Aragón\nCódigo postal: 44128\nEmail: u9kIJ@ual.es\nTeléfono fijo: +34 978 64 47 30\nTeléfono móvil: +34 678 69 16 51\nNHC: 6566985\nCondición de riesgo: Técnico de Laboratorio\n\nDatos asistenciales.\nMédico: Dr. Juan José Márquez Otero. NC 792000217. Residente de 4° año en Endocrinología. Instituto Maimónides de Investigación Biomédica de Córdoba (IMIBIC). Avenida Menéndez Pidal, s/n. 14004. Córdoba. España.\nFecha de ingreso: 13/04/2021\nEpisodio: 76196258\nHospital: Hospital Universitario La Paz\nMatrícula del coche: 3109HGC\nModelo: Toyota Corolla\nVIN: VSS174DQ75N979276\n\nInforme clínico del paciente:\nPaciente vegetariano de 1

### OTROS_NUM

#### Historias clínicas

In [95]:
## Expresión regular para HISTORIA CLINICA
regexp_hc_prefix = r'\bh(\.?)c(\.?)\s*[:\.]?\s*'
regexp_hc_numbers = r'[0-9]{3,}'

clinic_history_anonymizer = PrefixRegexAnnonimizer(
    prefix_regex=regexp_hc_prefix,
    content_regex=regexp_hc_numbers,
    re_flags=re.IGNORECASE,
    category='XOTROS_NUMX',
    output_type=OUTPUT_TYPE,
)

print(clinic_history_anonymizer.anonymize("Atendido previamente hc 23411, control prenatal semana 27"))
print(clinic_history_anonymizer.anonymize("Atendido previamente h.c. 23411, control prenatal semana 27"))
print(clinic_history_anonymizer.anonymize("Atendido  previamente hc h.c.: 23411 control prenatal semana 27"))
print(clinic_history_anonymizer.anonymize("Refiere extravio hc sin sintomatologia tel 27384234"))

Atendido previamente hc  XOTROS_NUMX, control prenatal semana 27
Atendido previamente h.c.  XOTROS_NUMX, control prenatal semana 27
Atendido  previamente hc h.c.:  XOTROS_NUMX control prenatal semana 27
Refiere extravio hc sin sintomatologia tel 27384234


In [96]:
documents['anonymized'] = documents['anonymized'].apply(clinic_history_anonymizer.anonymize)

In [97]:
documents[documents['anonymized'].str.contains('XOTROS_NUMX', case=False)]

,original,preprocessed,anonymized


### DIRECCIÓN

In [98]:
streets = pd.read_csv("datasets/diccionario_calles_editadas.csv", sep=';', index_col=None)
streets.head(5)

,categoria,nombre,provincia_nombre,Token0,Token1,Token2,Token3,Token4,Token5,Token6
0,PJE,pje estudiante,La Rioja,pje,estudiante,NaN,NaN,NaN,NaN,NaN
1,CALLE,20 de mayo,La Rioja,20,de,mayo,NaN,NaN,NaN,NaN
2,CALLE,1 de enero,La Rioja,1,de,enero,NaN,NaN,NaN,NaN
3,CALLE,10 de julio,La Rioja,10,de,julio,NaN,NaN,NaN,NaN
4,CALLE,aconcagua,La Rioja,aconcagua,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
# Sanitize street names
street_names = [remove_accents(name).strip()
                for name in streets.nombre
                if not re.search(ILLEGAL_CHAR_REGEX, name)]
regexp_address_prefix = r'\b(calle|avenida|av\.?|pasaje|pje\.?|parcela|manzana|tira|bloque|casa|lote|barrio)\s*[:]?\s*'
# Nombre de calles o barrios incluyendo cualquier numero posterior
regexp_streets = '(' + '|'.join(street_names) + ')(\s+([0-9]{1,6}))?'

address_w_names_anonymizer = PrefixRegexAnnonimizer(
    prefix_regex=regexp_address_prefix,
    content_regex=regexp_streets,
    re_flags=re.IGNORECASE,
    category='XDIRECCIONX',
    output_type=OUTPUT_TYPE,
)

# Ejemplos
print(address_w_names_anonymizer.anonymize("Vive en av. antartida argentina 1470, se indica medicacion"))
print(address_w_names_anonymizer.anonymize("Direccion: calle: 10 de julio, Barrio: alfonsina storini con hermanos"))
print(address_w_names_anonymizer.anonymize("Vive en Barrio barrioquenoexiste con hermanos"))
print(address_w_names_anonymizer.anonymize("Vive en manzana 3 lote 9 barrio 10 de julio."))

Vive en av.  XDIRECCIONX, se indica medicacion
Direccion: calle:  XDIRECCIONX, Barrio:  XDIRECCIONX con hermanos
Vive en Barrio barrioquenoexiste con hermanos
Vive en manzana 3 lote 9 barrio  XDIRECCIONX.


<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_63444/2484243500.py:7: SyntaxWarning: invalid escape sequence '\s'
  regexp_streets = '(' + '|'.join(street_names) + ')(\s+([0-9]{1,6}))?'


In [100]:
regexp_address_prefix = r'\b(calle|avenida|pasaje|pje\.?|parcela|manzana|tira|bloque|casa|lote|barrio)\s*[:]?\s*'
# Palabra o numero
regexp_address_content = r'\b([0-9]{1,6}|[a-zA-Z]+)\b'

address_anonymizer = PrefixRegexAnnonimizer(
    prefix_regex=regexp_address_prefix,
    content_regex=regexp_address_content,
    re_flags=re.IGNORECASE,
    category='XDIRECCIONX',
    output_type=OUTPUT_TYPE,
)

# Ejemplos
print(address_anonymizer.anonymize("Vive en avenida antartida argentina 1470, se indica medicacion"))
print(address_anonymizer.anonymize("Direccion: calle: 10 de julio, Barrio: alfonsina storini con hermanos"))
print(address_anonymizer.anonymize("Vive en Barrio barrioquenoexiste con hermanos"))
print(address_anonymizer.anonymize("Vive en manzana 3 lote 9 barrio 10 de julio."))

Vive en avenida  XDIRECCIONX argentina 1470, se indica medicacion
Direccion: calle:  XDIRECCIONX de julio, Barrio:  XDIRECCIONX storini con hermanos
Vive en Barrio  XDIRECCIONX con hermanos
Vive en manzana  XDIRECCIONX lote  XDIRECCIONX barrio  XDIRECCIONX de julio.


In [101]:
documents['anonymized'] = documents['anonymized'].apply(address_w_names_anonymizer.anonymize)
documents['anonymized'] = documents['anonymized'].apply(address_anonymizer.anonymize)

In [102]:
documents[documents['anonymized'].str.contains('XDIRECCIONX', case=False)]

,original,preprocessed,anonymized
0,"Datos del paciente.\nNombre: María Soledad Moreno Roca\nDNI: 23556552K\nFecha de nacimiento: 09/01/1941\nGénero: Mujer\nDomicilio: Calle de Almagro 80\nCiudad: Denia, Valencia, Comunidad Valenciana\nCódigo postal: 46571\nEmail: mariasoledad_roca@ucm.es\nTeléfono fijo: +34 960 66 89 48\nTeléfono móvil: +34 660 57 14 97\nNHC: 2409425\nNASS: 468043486571\nCondición de riesgo: Científico de Investigación\n\nDatos asistenciales.\nMédico: Dr. Juan Ramón Benito Vicente. NC 097900390. Investigador Clínico en Epidemiología. Instituto de Investigación Biomédica en Red de Enfermedades Infecciosas (CIBERINFEC). Avenida Monforte de Lemos 3-5. 28029. Madrid. España.\nFecha de ingreso: 05/06/1996\nCentro de salud: Centro de Salud Carabanchel\n\nInforme clínico del paciente:\nPaciente sobreviviente de violencia de 55 años de edad, acompañado de su madre.","Datos del paciente.\nNombre: Maria Soledad Moreno Roca\nDNI: 23556552K\nFecha de nacimiento: 09/01/1941\nGenero: Mujer\nDomicilio: Calle de Almagro 80\nCiudad: Denia, Valencia, Comunidad Valenciana\nCodigo postal: 46571\nEmail: mariasoledad_roca@ucm.es\nTelefono fijo: +34 960 66 89 48\nTelefono movil: +34 660 57 14 97\nNHC: 2409425\nNASS: 468043486571\nCondicion de riesgo: Cientifico de Investigacion\n\nDatos asistenciales.\nMedico: Dr. Juan Ramon Benito Vicente. NC 097900390. Investigador Clinico en Epidemiologia. Instituto de Investigacion Biomedica en Red de Enfermedades Infecciosas (CIBERINFEC). Avenida Monforte de Lemos 3-5. 28029. Madrid. España.\nFecha de ingreso: 05/06/1996\nCentro de salud: Centro de Salud Carabanchel\n\nInforme clinico del paciente:\nPaciente sobreviviente de violencia de 55 años de edad, acompañado de su madre.","Datos del paciente.\nNombre: Maria Soledad Moreno Roca\nDNI: 23556552K\nFecha de nacimiento: XFECHAX\nGenero: Mujer\nDomicilio: Calle XDIRECCIONX Almagro 80\nCiudad: Denia, Valencia, Comunidad Valenciana\nCodigo postal: 46571\nEmail: XCORREO_ELECTRONICOX\nTelefono fijo: +34 9XFECHAX 48\nTelefono movil: +34 6XFECHAX 97\nNHC: 2409425\nNASS: 468043486571\nCondicion de riesgo: Cientifico de Investigacion\n\nDatos asistenciales.\nMedico: Dr. Juan Ramon Benito Vicente. NC 097900390. Investigador Clinico en Epidemiologia. Instituto de Investigacion Biomedica en Red de Enfermedades Infecciosas (CIBERINFEC). Avenida XDIRECCIONX de Lemos 3-5. 28029. Madrid. España.\nFecha de ingreso: XFECHAX\nCentro de salud: Centro de Salud Carabanchel\n\nInforme clinico del paciente:\nPaciente sobreviviente de violencia de 55 años de edad, acompañado de su madre."
1,"Datos del paciente.\nNombre: Victoria Plaza Roca\nDNI: 75398682J\nFecha de nacimiento: 27/09/2014\nGénero: F\nDomicilio: Calle de Arganzuela 18\nCiudad: Gijón, La Rioja, La Rioja\nCódigo postal: 26167\nEmail: victoria_roca@ciberer.es\nTeléfono fijo: +34 941 29 92 15\nTeléfono móvil: +34 641 97 74 21\nNHC: 1980495\nNASS: 620412206059\nCondición de riesgo: Operador de Maquinaria Pesada\n\nDatos asistenciales.\nMédico: Dr. Eric Alarcon Pascual. NC 646335336. Neurólogo Investigador en Trastornos Cognitivos. Instituto de Investigación Sanitaria del Hospital 12 de Octubre (Imas12). Avenida de Córdoba, s/n. 28041. Madrid. España.\nFecha de ingreso: 21/10/2021\nEpisodio: 24822158\nCentro de salud: Centro de Salud Málaga Oeste\nMatrícula del coche: 5267NPH\nModelo: Mercedes-Benz C-Class\nVIN: VSL4HQZ1ZGI361218\n\nInforme clínico del paciente:\nPaciente de ascendencia australiana de 7 años de edad, acompañado de su tía. Se registró una sesión de escaneo del iris, el código obtenido para revisarla es KJ369UZN325.","Datos del paciente.\nNombre: Victoria Plaza Roca\nDNI: 75398682J\nFecha de nacimiento: 27/09/2014\nGenero: F\nDomicilio: Calle de Arganzuela 18\nCiudad: Gijon, La Rioja, La Rioja\nCodigo postal: 26167\nEmail: victoria_roca@ciberer.es\nTelefono fijo: +34 941 29 92 15\nTelefono movil: +34 641 97 74 21\nNHC: 1980495\nNASS: 620412206059\nCondicion de riesgo: Operador de Maquinaria Pe

### EFECTOR

Se siguen dos estrategias
- CAPS y Hospitales por diccionario con trigger words
- Siglas más usuales (HMN, HMyN, etc)

#### CAPS

In [103]:
caps = pd.read_csv("datasets/diccionario_caps.csv", sep=';', index_col=None)
caps.head(2)

,Nombre,Token0,Token1,Token2,Token3,Token4,Token5,Token6
0,caps 20 de mayo,caps,20,de,mayo,NaN,NaN,NaN
1,caps agua blanca,caps,agua,blanca,NaN,NaN,NaN,NaN


In [104]:
# Sanitize caps names
caps_names = [remove_accents(name).replace('caps', '').strip()
                for name in caps.Nombre
                if not re.search(ILLEGAL_CHAR_REGEX, name)]

regexp_caps_prefix = r'(CAPS|CPS)\s*[:\.]?'
regexp_caps_names = r'\b(' + '|'.join(caps_names) + r')\b'

caps_anonymizer = PrefixRegexAnnonimizer(
    prefix_regex=regexp_caps_prefix,
    content_regex=regexp_caps_names,
    re_flags=re.IGNORECASE,
    category='XCAPSX',
    output_type=OUTPUT_TYPE,
)

print(caps_anonymizer.anonymize("Atendido previamente en caps 20 de mayo, se indica medicacion"))
print(caps_anonymizer.anonymize("Atendido previamente en caps: agua blanca"))
print(caps_anonymizer.anonymize("Atendido previamente en cps: Agua Blanca"))
print(caps_anonymizer.anonymize("Atendido previamente en caps noexiste"))

Atendido previamente en caps  XCAPSX, se indica medicacion
Atendido previamente en caps: XCAPSX
Atendido previamente en cps: XCAPSX
Atendido previamente en caps noexiste


In [105]:
documents['anonymized'] = documents['anonymized'].apply(caps_anonymizer.anonymize)

In [106]:
documents[documents['anonymized'].str.contains('XCAPSX', case=False)]

,original,preprocessed,anonymized


#### Hospitales

In [107]:
hospitals = pd.read_csv("datasets/diccionario_hospitales.csv", sep=';', index_col=None)
hospitals.head(2)

,Nombre,Token0,Token1,Token2,Token3,Token4,Token5,Token6,Token7,Token8,Token9,Token10
0,hospital de la madre y el niño (la rioja) - capital,hospital,de,la,madre,y,el,niño,(la,rioja),-,capital
1,hospital campanas,hospital,campanas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
# Sanitize hospital names
hospitals_names = [
    remove_accents(name).replace('hospital', '').split('-')[0].split('(')[0].strip()
    for name in hospitals.Nombre
]
hospitals_names = [
    name for name in hospitals_names
    if not re.search(ILLEGAL_CHAR_REGEX, name)
]

regexp_hospital_prefix = r'Hospital|Hosp\.?|H(\.)?\s*[:\.]?'
regexp_hospital_names = r'\b(' + '|'.join(hospitals_names) + r')\b'
regexp_hospital_names = regexp_hospital_names.replace('.', '\.?')

hospital_anonymizer = PrefixRegexAnnonimizer(
    prefix_regex=regexp_hospital_prefix,
    content_regex=regexp_hospital_names,
    re_flags=re.IGNORECASE,
    category='XHOSPX',
    output_type=OUTPUT_TYPE,
)

print(hospital_anonymizer.anonymize("Atendido previamente en hospital segundo b. romero, se indica medicacion"))
print(hospital_anonymizer.anonymize("Atendido previamente en hospital segundo b romero, se indica medicacion"))
print(hospital_anonymizer.anonymize("Atendido previamente en hosp villa castelli"))
print(hospital_anonymizer.anonymize("Atendido previamente en H  Villa Castelli"))
print(hospital_anonymizer.anonymize("Atendido previamente en hosp por la maniana"))

Atendido previamente en hospital XHOSPX, se indica medicacion
Atendido previamente en hospital XHOSPX, se indica medicacion
Atendido previamente en hosp XHOSPX
Atendido previamente en H   XHOSPX
Atendido previamente en hosp por la maniana


<>:13: SyntaxWarning: invalid escape sequence '\.'
<>:13: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipykernel_63444/1765221295.py:13: SyntaxWarning: invalid escape sequence '\.'
  regexp_hospital_names = regexp_hospital_names.replace('.', '\.?')


In [109]:
documents['anonymized'] = documents['anonymized'].apply(hospital_anonymizer.anonymize)

In [110]:
documents[documents['anonymized'].str.contains('XHOSPX', case=False)]

,original,preprocessed,anonymized


### INSTITUCIÓN

In [111]:
schools = pd.read_csv("datasets/diccionario_est_educativos.csv", sep=';', index_col=None)
schools.head(2)

,Nombre,Tipo,Numeración,Nombre_limpio,Token0,Token1,Token2,Token3,Token4,Token5,Token6,Token7,Token8,Token9,Token10,Token11,Token12,Token13,Token14
0,esc.n°230 bajo hondo,esc.,n°230,bajo hondo,bajo,hondo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,esc.n°353 batallon de ing. de construcciones 141,esc.,n°353,batallon de ing. de construcciones 141,batallon,de,ing.,de,construcciones,141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [112]:
# Sanitize school names
# Remove content in ()
school_names = [
    re.sub(r'\(.*?\)', '', name).replace('(', '').replace(')', '').strip()
    for name in schools.Nombre_limpio if isinstance(name, str)
]
school_names = set([
    remove_accents(name)
    for name in school_names if len(name) > 1 and not re.search(ILLEGAL_CHAR_REGEX, name)
])

regexp_school_main = r'escuela|esc\.|esc|colegio|jardin|infantes|instituto'
regexp_school_detail = r'primari[ao]|normal|secundari[ao]|privad[ao]|maternal|especial'
regexp_school_number = r'(\sn\s?\.?\°?)?\s*[0-9]+'
regexp_school_prefix = rf'({regexp_school_main})(?:\s+(?:{regexp_school_detail}))?'

regexp_school_names = '|'.join(school_names)
regexp_school_names = regexp_school_names.replace('.', '\.?')
regexp_school_names = rf'({regexp_school_number}\s?)?({regexp_school_names})'

# Anonimiza el nombre de los establecimientos
school_name_anonymizer = PrefixRegexAnnonimizer(
    prefix_regex=regexp_school_prefix,
    content_regex=regexp_school_names,
    re_flags=re.IGNORECASE,
    category='XINSTITUCIONX',
    output_type=OUTPUT_TYPE,
)

print(school_name_anonymizer.anonymize("Asiste a jardin maternal rincon de luz."))
print(school_name_anonymizer.anonymize("Asiste a colegio bajo hondo."))
print(school_name_anonymizer.anonymize("Asiste a colegio secundario bajo hondo."))
print(school_name_anonymizer.anonymize("Asiste a esc normal dr. jose santos salinas por la tarde"))
print(school_name_anonymizer.anonymize("Asiste a esc normal inexistente"))
print(school_name_anonymizer.anonymize("Asiste a esc n 345 munic.beatriz de la v.de costa medina"))
print(school_name_anonymizer.anonymize("Asiste a Esc. n° 345 para el paicor"))

Asiste a jardin maternal XINSTITUCIONX.
Asiste a colegio XINSTITUCIONX.
Asiste a colegio secundario XINSTITUCIONX.
Asiste a esc XINSTITUCIONX por la tarde
Asiste a esc normal inexistente
Asiste a esc XINSTITUCIONX
Asiste a Esc. n° 345 para el paicor


<>:18: SyntaxWarning: invalid escape sequence '\.'
<>:18: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipykernel_63444/1147442673.py:18: SyntaxWarning: invalid escape sequence '\.'
  regexp_school_names = regexp_school_names.replace('.', '\.?')


In [113]:
# Anonimiza solo el numero
school_number_anonymizer = PrefixRegexAnnonimizer(
    prefix_regex=regexp_school_prefix,
    content_regex=regexp_school_number,
    re_flags=re.IGNORECASE,
    category='XINSTITUCIONX',
    output_type=OUTPUT_TYPE,
)

print(school_number_anonymizer.anonymize("Asiste a esc normal inexistente"))
print(school_number_anonymizer.anonymize("Asiste a esc n 345 munic.beatriz de la v.de costa medina"))
print(school_number_anonymizer.anonymize("Asiste a Esc. n° 345 para el paicor"))

Asiste a esc normal inexistente
Asiste a esc XINSTITUCIONX munic.beatriz de la v.de costa medina
Asiste a Esc. XINSTITUCIONX para el paicor


In [114]:
documents['anonymized'] = documents['anonymized'].apply(school_name_anonymizer.anonymize)
documents['anonymized'] = documents['anonymized'].apply(school_number_anonymizer.anonymize)

In [115]:
documents[documents['anonymized'].str.contains('XINSTITUCIONX', case=False)]

,original,preprocessed,anonymized


#### Grados

In [116]:
# regexp_grado = re.compile(r'^grado$', re.IGNORECASE)
# regexp_grado_letras = re.compile(r'^primer$|^segundo$|^tercer$|^cuarto$|^quinto$|^sexto$|^septimo$', re.IGNORECASE)
# regexp_grado_combinado = re.compile(r'[0-9]{1}[a-z]{1,3}|[0-9]{1}º', re.IGNORECASE)
# regexp_grado_exclusión = re.compile(r'obesidad|quemadura|evoluci[oóò]n', re.IGNORECASE)

# def deidentificar_grado(texto_a_deidentificar):
#     lista_texto_a_deidentificar=texto_a_deidentificar.split(' ')
    
#     ## Iteración entre tokens de la lista del texto a deidentificar
#     for j, token in enumerate(lista_texto_a_deidentificar):
    
#         # Detecta si el token actual es grado
#         if regexp_grado.search(lista_texto_a_deidentificar[j]):
#             #Excluyo grados de obesidad y quemaduras
#             if not regexp_grado_exclusión.search(lista_texto_a_deidentificar[j-3]) or regexp_grado_exclusión.search(lista_texto_a_deidentificar[j-2]):
#                 if regexp_grado_letras.search(lista_texto_a_deidentificar[j-1]) or regexp_grado_combinado.search(lista_texto_a_deidentificar[j-1]):
#                 # Reemplazar el token actual por el patrón correspondiente
#                     if output_tipo==1:
#                         lista_texto_a_deidentificar[j] = '<'+ lista_texto_a_deidentificar[j] +'>'
#                         lista_texto_a_deidentificar[j-1] = '<'+ lista_texto_a_deidentificar[j-1] +'>'
#                     if output_tipo==2:
#                         lista_texto_a_deidentificar[j] = 'XXXX'
#                         lista_texto_a_deidentificar[j-1] = 'XXXX'
#                     if output_tipo==3:
#                         lista_texto_a_deidentificar[j] = 'XGRADOX'
#                         lista_texto_a_deidentificar[j-1] = 'XGRADOX'
    
#     # Generar el texto deidentificado concatenando todos los tokens ya revisados (y modificados si corrrespondiera)
#     return ' '.join(lista_texto_a_deidentificar)
    
# textos_a_deidentificar['texto_anonimizado'] = textos_a_deidentificar['texto_anonimizado'].apply(deidentificar_grado)

#### Turnos

In [117]:
## Expresión regular para TURNOS
regexp_shcool_turn = r'\b(turno mañana|turno tarde|TT|TM)\b'

shcool_turn_anonymizer = RegexAnnonimizer(
    content_regex=regexp_shcool_turn,
    category='XTURNOX',
    re_flags=re.IGNORECASE,
    output_type=OUTPUT_TYPE,
)

print(shcool_turn_anonymizer.anonymize("Asiste a esc en turno tarde"))
print(shcool_turn_anonymizer.anonymize("Asiste a esc sexto grado TT por la mañana,"))
print(shcool_turn_anonymizer.anonymize("Asiste a esc sexto grado turno mañana."))

Asiste a esc en XTURNOX
Asiste a esc sexto grado XTURNOX por la mañana,
Asiste a esc sexto grado XTURNOX.


In [118]:
documents['anonymized'] = documents['anonymized'].apply(school_name_anonymizer.anonymize)

In [119]:
documents[documents['anonymized'].str.contains('XTURNOX', case=False)]

,original,preprocessed,anonymized


### ZONA

In [120]:
neighborhood = pd.read_csv("datasets/diccionario_barrios_populares.csv", sep=';')
province = pd.read_csv("datasets/diccionario_provincias.csv", sep=';')
city = pd.read_csv("datasets/diccionario_localidades.csv", sep=';')
county = pd.read_csv("datasets/diccionario_municipios.csv", sep=';')

county.head()

,ciudad,Token0,Token1,Token2,Token3,Token4
0,aimogasta,aimogasta,NaN,NaN,NaN,NaN
1,arauco,arauco,NaN,NaN,NaN,NaN
2,aminga,aminga,NaN,NaN,NaN,NaN
3,castro barros,castro,barros,NaN,NaN,NaN
4,chamical,chamical,NaN,NaN,NaN,NaN


In [121]:
location_names = (
    neighborhood['nombre'].values.tolist() + county['ciudad'].values.tolist() +
    city['nombre'].values.tolist() + province['provincia_nombre'].values.tolist()
)

location_names = set([
    remove_accents(name)
    for name in location_names if len(name) > 1 and not re.search(ILLEGAL_CHAR_REGEX, name)
])

regexp_locations = r'\b(' + '|'.join(location_names) + r')\b'

location_anonymizer = RegexAnnonimizer(
    content_regex=regexp_locations,
    category='XZONAX',
    re_flags=re.IGNORECASE,
    output_type=OUTPUT_TYPE,
)

print(location_anonymizer.anonymize("Paciente se traslada desde Chamical para consulta"))
print(location_anonymizer.anonymize("Paciente se traslada desde CASTRO BARROS, para consulta"))

Paciente se traslada desde XZONAX para consulta
Paciente se traslada desde XZONAX, para consulta


In [122]:
documents['anonymized'] = documents['anonymized'].apply(location_anonymizer.anonymize)

In [123]:
documents[documents['anonymized'].str.contains('XZONAX', case=False)]

,original,preprocessed,anonymized
1,"Datos del paciente.\nNombre: Victoria Plaza Roca\nDNI: 75398682J\nFecha de nacimiento: 27/09/2014\nGénero: F\nDomicilio: Calle de Arganzuela 18\nCiudad: Gijón, La Rioja, La Rioja\nCódigo postal: 26167\nEmail: victoria_roca@ciberer.es\nTeléfono fijo: +34 941 29 92 15\nTeléfono móvil: +34 641 97 74 21\nNHC: 1980495\nNASS: 620412206059\nCondición de riesgo: Operador de Maquinaria Pesada\n\nDatos asistenciales.\nMédico: Dr. Eric Alarcon Pascual. NC 646335336. Neurólogo Investigador en Trastornos Cognitivos. Instituto de Investigación Sanitaria del Hospital 12 de Octubre (Imas12). Avenida de Córdoba, s/n. 28041. Madrid. España.\nFecha de ingreso: 21/10/2021\nEpisodio: 24822158\nCentro de salud: Centro de Salud Málaga Oeste\nMatrícula del coche: 5267NPH\nModelo: Mercedes-Benz C-Class\nVIN: VSL4HQZ1ZGI361218\n\nInforme clínico del paciente:\nPaciente de ascendencia australiana de 7 años de edad, acompañado de su tía. Se registró una sesión de escaneo del iris, el código obtenido para revisarla es KJ369UZN325.","Datos del paciente.\nNombre: Victoria Plaza Roca\nDNI: 75398682J\nFecha de nacimiento: 27/09/2014\nGenero: F\nDomicilio: Calle de Arganzuela 18\nCiudad: Gijon, La Rioja, La Rioja\nCodigo postal: 26167\nEmail: victoria_roca@ciberer.es\nTelefono fijo: +34 941 29 92 15\nTelefono movil: +34 641 97 74 21\nNHC: 1980495\nNASS: 620412206059\nCondicion de riesgo: Operador de Maquinaria Pesada\n\nDatos asistenciales.\nMedico: Dr. Eric Alarcon Pascual. NC 646335336. Neurologo Investigador en Trastornos Cognitivos. Instituto de Investigacion Sanitaria del Hospital 12 de Octubre (Imas12). Avenida de Cordoba, s/n. 28041. Madrid. España.\nFecha de ingreso: 21/10/2021\nEpisodio: 24822158\nCentro de salud: Centro de Salud Malaga Oeste\nMatricula del coche: 5267NPH\nModelo: Mercedes-Benz C-Class\nVIN: VSL4HQZ1ZGI361218\n\nInforme clinico del paciente:\nPaciente de ascendencia australiana de 7 años de edad, acompañado de su tia. Se registro una sesion de escaneo del iris, el codigo obtenido para revisarla es KJ369UZN325.","Datos del paciente.\nNombre: Victoria Plaza Roca\nDNI: 75398682J\nFecha de nacimiento: XFECHAX\nGenero: F\nDomicilio: Calle XDIRECCIONX Arganzuela 18\nCiudad: Gijon, XZONAX, XZONAX\nCodigo postal: 26167\nEmail: XCORREO_ELECTRONICOX\nTelefono fijo: +34 9XFECHAX 15\nTelefono movil: +34 6XFECHAX 21\nNHC: 1980495\nNASS: 620412206059\nCondicion de riesgo: Operador de Maquinaria Pesada\n\nDatos asistenciales.\nMedico: Dr. Eric Alarcon Pascual. NC 646335336. Neurologo Investigador en Trastornos Cognitivos. Instituto de Investigacion Sanitaria del Hospital 12 de XFECHAX (Imas12). Avenida XDIRECCIONX XZONAX, s/n. 28041. Madrid. España.\nFecha de ingreso: XFECHAX\nEpisodio: 24822158\nCentro de salud: Centro de Salud Malaga Oeste\nMatricula del coche: 5267NPH\nModelo: Mercedes-Benz C-Class\nVIN: VSL4HQZ1ZGI361218\n\nInforme clinico del paciente:\nPaciente de ascendencia australiana de 7 años de edad, acompañado de su tia. Se registro una sesion de escaneo del iris, el codigo obtenido para revisarla es KJ369UZN325."
2,"Datos del paciente.\nNombre: Iñigo Escobar Rivero\nDNI: 97790373P\nFecha de nacimiento: 08/09/1944\nGénero: Masculino\nDomicilio: Calle de los Embajadores 88\nCiudad: Telde, Gerona, Cataluña\nCódigo postal: 17012\nEmail: inigo-escobar@imibic.org\nTeléfono fijo: +34 972 65 06 71\nTeléfono móvil: +34 772 38 57 17\nNHC: 8245120\nNASS: 724651062765\nCondición de riesgo: Trabajador de Sanitización\n\nDatos asistenciales.\nMédico: Dra. Emma Vicente Santos. NC 588434040. Neuróloga Investigadora en Trastornos Cognitivos. Instituto de Investigación Sanitaria del Hospital 12 de Octubre (Imas12). Avenida de Córdoba, s/n. 28041. Madrid. España.\nFecha de ingreso: 24/02/1975\nEpisodio: 00580872\nCentro de salud: Centro de Salud Chamartín\n\nInforme clínico del paciente:\nPaciente migrante de 30 años de edad.","Datos del paciente.\nNombre: Iñigo Escobar Rivero\nDNI: 97790373P\nFecha de nacimiento: 08/

### PAÍS

Preparación del listado de paises a detectar:

In [124]:
countries = pd.read_csv("datasets/diccionario_paises.csv", sep=';', index_col=None)

# Filtro algunas entradas teniendo en cuenta que su uso como país es muy infrecuente en el nuestro.
p1 = re.compile(
    'Nueva|Reunion|San Martin|Islas Cocos \(Keeling\)|india|malta|montenegro|guadalupe|mauricio|laos|'
    'eritrea|iran|polonia|portugal|camboya|islas malvinas|antartida|argentina',
    re.IGNORECASE)

countries = countries[~countries['nombre'].str.contains(p1)]
countries.tail()

<>:5: SyntaxWarning: invalid escape sequence '\('
<>:5: SyntaxWarning: invalid escape sequence '\('
/tmp/ipykernel_63444/737399818.py:5: SyntaxWarning: invalid escape sequence '\('
  'Nueva|Reunion|San Martin|Islas Cocos \(Keeling\)|india|malta|montenegro|guadalupe|mauricio|laos|'


,nombre,Token0,Token1,Token2,Token3,Token4,Token5,Token6,Token7
243,wallis y futuna,wallis,y,futuna,NaN,NaN,NaN,NaN,NaN
244,yemen,yemen,NaN,NaN,NaN,NaN,NaN,NaN,NaN
245,yibuti,yibuti,NaN,NaN,NaN,NaN,NaN,NaN,NaN
246,zambia,zambia,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,zimbabue,zimbabue,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [125]:
country_names = set([
    remove_accents(name).strip()
    for name in countries.nombre if len(name) > 1 and not re.search(ILLEGAL_CHAR_REGEX, name)
])

regexp_countries = r'\b(' + r'|'.join(country_names) + r')\b'

country_anonymizer = RegexAnnonimizer(
    content_regex=regexp_countries,
    category='XPAISX',
    re_flags=re.IGNORECASE,
    output_type=OUTPUT_TYPE,
)

print(country_anonymizer.anonymize("Paciente se traslada desde ZAMBIA para consulta"))
print(country_anonymizer.anonymize("Paciente se traslada desde wallis y futuna, para consulta"))

Paciente se traslada desde XPAISX para consulta
Paciente se traslada desde XPAISX, para consulta


In [126]:
documents['anonymized'] = documents['anonymized'].apply(country_anonymizer.anonymize)

In [127]:
documents[documents['anonymized'].str.contains('XPAISX', case=False)]

,original,preprocessed,anonymized
0,"Datos del paciente.\nNombre: María Soledad Moreno Roca\nDNI: 23556552K\nFecha de nacimiento: 09/01/1941\nGénero: Mujer\nDomicilio: Calle de Almagro 80\nCiudad: Denia, Valencia, Comunidad Valenciana\nCódigo postal: 46571\nEmail: mariasoledad_roca@ucm.es\nTeléfono fijo: +34 960 66 89 48\nTeléfono móvil: +34 660 57 14 97\nNHC: 2409425\nNASS: 468043486571\nCondición de riesgo: Científico de Investigación\n\nDatos asistenciales.\nMédico: Dr. Juan Ramón Benito Vicente. NC 097900390. Investigador Clínico en Epidemiología. Instituto de Investigación Biomédica en Red de Enfermedades Infecciosas (CIBERINFEC). Avenida Monforte de Lemos 3-5. 28029. Madrid. España.\nFecha de ingreso: 05/06/1996\nCentro de salud: Centro de Salud Carabanchel\n\nInforme clínico del paciente:\nPaciente sobreviviente de violencia de 55 años de edad, acompañado de su madre.","Datos del paciente.\nNombre: Maria Soledad Moreno Roca\nDNI: 23556552K\nFecha de nacimiento: 09/01/1941\nGenero: Mujer\nDomicilio: Calle de Almagro 80\nCiudad: Denia, Valencia, Comunidad Valenciana\nCodigo postal: 46571\nEmail: mariasoledad_roca@ucm.es\nTelefono fijo: +34 960 66 89 48\nTelefono movil: +34 660 57 14 97\nNHC: 2409425\nNASS: 468043486571\nCondicion de riesgo: Cientifico de Investigacion\n\nDatos asistenciales.\nMedico: Dr. Juan Ramon Benito Vicente. NC 097900390. Investigador Clinico en Epidemiologia. Instituto de Investigacion Biomedica en Red de Enfermedades Infecciosas (CIBERINFEC). Avenida Monforte de Lemos 3-5. 28029. Madrid. España.\nFecha de ingreso: 05/06/1996\nCentro de salud: Centro de Salud Carabanchel\n\nInforme clinico del paciente:\nPaciente sobreviviente de violencia de 55 años de edad, acompañado de su madre.","Datos del paciente.\nNombre: Maria Soledad Moreno Roca\nDNI: 23556552K\nFecha de nacimiento: XFECHAX\nGenero: Mujer\nDomicilio: Calle XDIRECCIONX Almagro 80\nCiudad: Denia, Valencia, Comunidad Valenciana\nCodigo postal: 46571\nEmail: XCORREO_ELECTRONICOX\nTelefono fijo: +34 9XFECHAX 48\nTelefono movil: +34 6XFECHAX 97\nNHC: 2409425\nNASS: 468043486571\nCondicion de riesgo: Cientifico de Investigacion\n\nDatos asistenciales.\nMedico: Dr. Juan Ramon Benito Vicente. NC 097900390. Investigador Clinico en Epidemiologia. Instituto de Investigacion Biomedica en Red de Enfermedades Infecciosas (CIBERINFEC). Avenida XDIRECCIONX de Lemos 3-5. 28029. Madrid. XPAISX.\nFecha de ingreso: XFECHAX\nCentro de salud: Centro de Salud Carabanchel\n\nInforme clinico del paciente:\nPaciente sobreviviente de violencia de 55 años de edad, acompañado de su madre."
1,"Datos del paciente.\nNombre: Victoria Plaza Roca\nDNI: 75398682J\nFecha de nacimiento: 27/09/2014\nGénero: F\nDomicilio: Calle de Arganzuela 18\nCiudad: Gijón, La Rioja, La Rioja\nCódigo postal: 26167\nEmail: victoria_roca@ciberer.es\nTeléfono fijo: +34 941 29 92 15\nTeléfono móvil: +34 641 97 74 21\nNHC: 1980495\nNASS: 620412206059\nCondición de riesgo: Operador de Maquinaria Pesada\n\nDatos asistenciales.\nMédico: Dr. Eric Alarcon Pascual. NC 646335336. Neurólogo Investigador en Trastornos Cognitivos. Instituto de Investigación Sanitaria del Hospital 12 de Octubre (Imas12). Avenida de Córdoba, s/n. 28041. Madrid. España.\nFecha de ingreso: 21/10/2021\nEpisodio: 24822158\nCentro de salud: Centro de Salud Málaga Oeste\nMatrícula del coche: 5267NPH\nModelo: Mercedes-Benz C-Class\nVIN: VSL4HQZ1ZGI361218\n\nInforme clínico del paciente:\nPaciente de ascendencia australiana de 7 años de edad, acompañado de su tía. Se registró una sesión de escaneo del iris, el código obtenido para revisarla es KJ369UZN325.","Datos del paciente.\nNombre: Victoria Plaza Roca\nDNI: 75398682J\nFecha de nacimiento: 27/09/2014\nGenero: F\nDomicilio: Calle de Arganzuela 18\nCiudad: Gijon, La Rioja, La Rioja\nCodigo postal: 26167\nEmail: victoria_roca@ciberer.es\nTelefono fijo: +34 941 29 92 15\nTelefono movil: +34 641 97 74 21\nNHC: 1980495\nNASS: 620412206059\nCondicion de riesgo: Operador de Maquinaria Pe

### PERSONA

Chequea token a token si:

- ~~comienza con mayúscual y sigue con minúsculas~~
- es un nombre,
- es un apellido

In [128]:
## Lista de nombres en minúscula, sin números, con al menos 3 letras, sin stop Words y con frecuencia mayor a 5000.
first_names = pd.read_csv("datasets/nombres_editados.csv", sep=';', index_col = None)['nombre_limpio'].to_list()
last_names = pd.read_csv("datasets/apellidos_unicos.csv", sep=';', index_col = None)['apellido'].to_list()
names = [
    remove_accents(name).strip()
    for name in first_names + last_names
    if isinstance(name, str) and len(name) > 1 and not re.search(ILLEGAL_CHAR_REGEX, name)
]
names[:5]

['aaron', 'abel', 'abelardo', 'abigail', 'abraham']

In [129]:
len(set(names))

75911

In [130]:
# Exclude common noun names from list
noun_names = [
    'papa', 'tamaño', 'salud', 'hospital', 'niño', 'medico', 'toma', 'fecha', 'rodilla', 'vigil',
    'abajo', 'alegre', 'abusar', 'cola', 'cancela', 'grande', 'cortes', 'saliva', 'sola', 
    'areas', 'bajo', 'bien', 'blando', 'bota', 'buena', 'pareja', 'claro', 'claros', 'colica', 
    'cornea', 'cristalino', 'agua', 'corto', 'tiempo', 'grado', 'dieta', 'entrada', 'esposo', 
    'izquierdo', 'genera', 'general', 'genero', 'grados', 'haber', 'hierro', 'tres', 'igual', 
    'indico', 'plano', 'ligero', 'mañana', 'medica', 'medio', 'mucha', 'nacer', 'noto', 
    'parejas', 'pares', 'parto', 'pene', 'peso', 'piernas', 'requiere', 'semana', 'vaginal', 
    'varon', 'vino', 'nacimiento', 'hitos', 'menor', 'cordon', 'cuello', 'guardia', 'muñeca',
    'factor', 'chagas', 'sano', 'camino', 'manera', 'signo', 'come', 'solo', 'sola', 'cera',
    'tapon', 'cabeza', 'base', 'lenta', 'patron', 'borde', 'abuela', 'alli', 'alto', 'amigo',
    'anillo', 'aporta', 'area', 'arriba', 'bajar', 'balon', 'barrio', 'beber', 'boca',
    'brindo', 'bueno', 'busca', 'calle', 'calores', 'campo', 'cara', 'cargo', 'casa',
    'casi', 'caso', 'causa', 'cayo', 'cece', 'cena', 'chica', 'chico', 'constante',
    'cuadro', 'cuesta', 'debi', 'debia', 'deseo', 'deseo', 'DESEO', 'dias', 'dios',
    'done', 'dura', 'duro', 'escalera', 'estudia', 'figura', 'foco', 'Fondo', 'fosas',
    'fuertes', 'gato', 'gotas', 'guarda', 'hacen', 'haga', 'Jardin', 'labo', 'lado', 'lava',
    'legal', 'ligo', 'mala', 'mando', 'mano', 'mares', 'matar', 'mayor', 'medicina',
    'mesas', 'mori', 'mover', 'nace', 'nota', 'nuevo', 'pala', 'panico', 'pared', 'paro',
    'placer', 'presentado', 'presta', 'primo', 'publico', 'quiero', 'quizas', 'raiz',
    'redes', 'respeto', 'riesgo', 'rojo', 'rota', 'ruta', 'saber', 'sabia', 'sale',
    'sangra', 'segundo', 'segura', 'seis', 'severo', 'sobrino', 'soma', 'sule', 'teme',
    'terceros', 'tira', 'tirado', 'torno', 'trabajo', 'tranquilo', 'valor', 'vaya',
    'vender', 'vida', 'vital', 'vive', 'vivian', 'vuelo', 'comer', 'fuerte', 'doble', 'sorda',
    'numero', 'dolores', 'rostro', 'paterno', 'colon', 'diab', 'corte', 'crema',
    'bazo', 'camara', 'rabia', 'fuma', 'calcio', 'indice', 'timpano', 'mandato', 'escuela',
    'venia', 'vecino', 'terreno', 'sordo', 'salia', 'sala', 'saco', 'regalo', 'reales',
    'real', 'rato', 'rama', 'puerta', 'plata', 'pase', 'molares', 'llano', 'feto',
    'espinal', 'decia', 'cosa', 'carta', 'carne', 'cabello', 'brusco', 'bordes', 'bases',
    'baño', 'baba', 'avanza', 'amor', 'labio', 'maligno', 'teta'
]

names = set([name for name in names if name not in noun_names])
len(set(names))

75654

In [131]:
regexp_names = r'\b({})\b'.format('|'.join(names))

name_anonymizer = RegexAnnonimizer(
    content_regex=regexp_names,
    category='XPERSONAX',
    re_flags=re.IGNORECASE,
    output_type=OUTPUT_TYPE,
    verbose=True
)

print(name_anonymizer.anonymize("Paciente con tumor de gran tamaño medico con ibuprofeno."))
print(name_anonymizer.anonymize("Caida del cordon umbilical."))
print(name_anonymizer.anonymize("Acude a consulta con hermano Abel."))

Paciente con tumor de gran tamaño medico con ibuprofeno.
Caida del cordon umbilical.
Replacing Abel
Acude a consulta con hermano XPERSONAX.


In [132]:
documents['anonymized'] = documents['anonymized'].apply(name_anonymizer.anonymize)

Replacing Maria
Replacing Soledad
Replacing Moreno
Replacing Roca
Replacing Almagro
Replacing Ciudad
Replacing Valencia
Replacing NASS
Replacing Juan
Replacing Ramon
Replacing Benito
Replacing Vicente
Replacing Lemos
Replacing Madrid
Replacing Victoria
Replacing Plaza
Replacing Roca
Replacing Ciudad
Replacing Gijon
Replacing NASS
Replacing Eric
Replacing Alarcon
Replacing Pascual
Replacing Madrid
Replacing Malaga
Replacing coche
Replacing Mercedes
Replacing Benz
Replacing iris
Replacing Iñigo
Replacing Escobar
Replacing Rivero
Replacing Ciudad
Replacing NASS
Replacing Emma
Replacing Vicente
Replacing Santos
Replacing Madrid
Replacing Manuela
Replacing Morales
Replacing Cabrera
Replacing Ciudad
Replacing Elda
Replacing Teruel
Replacing Aragon
Replacing Susana
Replacing Carmona
Replacing Pidal
Replacing Alejandro
Replacing Galvez
Replacing Luna
Replacing Ciudad
Replacing Palma
Replacing Islas
Replacing NASS
Replacing Omar
Replacing Guerrero
Replacing Maldonado
Replacing Lemos
Replacing M

In [133]:
documents[documents['anonymized'].str.contains('XPERSONAX', case=False)]

,original,preprocessed,anonymized
0,"Datos del paciente.\nNombre: María Soledad Moreno Roca\nDNI: 23556552K\nFecha de nacimiento: 09/01/1941\nGénero: Mujer\nDomicilio: Calle de Almagro 80\nCiudad: Denia, Valencia, Comunidad Valenciana\nCódigo postal: 46571\nEmail: mariasoledad_roca@ucm.es\nTeléfono fijo: +34 960 66 89 48\nTeléfono móvil: +34 660 57 14 97\nNHC: 2409425\nNASS: 468043486571\nCondición de riesgo: Científico de Investigación\n\nDatos asistenciales.\nMédico: Dr. Juan Ramón Benito Vicente. NC 097900390. Investigador Clínico en Epidemiología. Instituto de Investigación Biomédica en Red de Enfermedades Infecciosas (CIBERINFEC). Avenida Monforte de Lemos 3-5. 28029. Madrid. España.\nFecha de ingreso: 05/06/1996\nCentro de salud: Centro de Salud Carabanchel\n\nInforme clínico del paciente:\nPaciente sobreviviente de violencia de 55 años de edad, acompañado de su madre.","Datos del paciente.\nNombre: Maria Soledad Moreno Roca\nDNI: 23556552K\nFecha de nacimiento: 09/01/1941\nGenero: Mujer\nDomicilio: Calle de Almagro 80\nCiudad: Denia, Valencia, Comunidad Valenciana\nCodigo postal: 46571\nEmail: mariasoledad_roca@ucm.es\nTelefono fijo: +34 960 66 89 48\nTelefono movil: +34 660 57 14 97\nNHC: 2409425\nNASS: 468043486571\nCondicion de riesgo: Cientifico de Investigacion\n\nDatos asistenciales.\nMedico: Dr. Juan Ramon Benito Vicente. NC 097900390. Investigador Clinico en Epidemiologia. Instituto de Investigacion Biomedica en Red de Enfermedades Infecciosas (CIBERINFEC). Avenida Monforte de Lemos 3-5. 28029. Madrid. España.\nFecha de ingreso: 05/06/1996\nCentro de salud: Centro de Salud Carabanchel\n\nInforme clinico del paciente:\nPaciente sobreviviente de violencia de 55 años de edad, acompañado de su madre.","Datos del paciente.\nNombre: XPERSONAX\nDNI: 23556552K\nFecha de nacimiento: XFECHAX\nGenero: Mujer\nDomicilio: Calle XDIRECCIONX XPERSONAX 80\nXPERSONAX: Denia, XPERSONAX, Comunidad Valenciana\nCodigo postal: 46571\nEmail: XCORREO_ELECTRONICOX\nTelefono fijo: +34 9XFECHAX 48\nTelefono movil: +34 6XFECHAX 97\nNHC: 2409425\nXPERSONAX: 468043486571\nCondicion de riesgo: Cientifico de Investigacion\n\nDatos asistenciales.\nMedico: Dr. XPERSONAX. NC 097900390. Investigador Clinico en Epidemiologia. Instituto de Investigacion Biomedica en Red de Enfermedades Infecciosas (CIBERINFEC). Avenida XDIRECCIONX de XPERSONAX 3-5. 28029. XPERSONAX. XPAISX.\nFecha de ingreso: XFECHAX\nCentro de salud: Centro de Salud Carabanchel\n\nInforme clinico del paciente:\nPaciente sobreviviente de violencia de 55 años de edad, acompañado de su madre."
1,"Datos del paciente.\nNombre: Victoria Plaza Roca\nDNI: 75398682J\nFecha de nacimiento: 27/09/2014\nGénero: F\nDomicilio: Calle de Arganzuela 18\nCiudad: Gijón, La Rioja, La Rioja\nCódigo postal: 26167\nEmail: victoria_roca@ciberer.es\nTeléfono fijo: +34 941 29 92 15\nTeléfono móvil: +34 641 97 74 21\nNHC: 1980495\nNASS: 620412206059\nCondición de riesgo: Operador de Maquinaria Pesada\n\nDatos asistenciales.\nMédico: Dr. Eric Alarcon Pascual. NC 646335336. Neurólogo Investigador en Trastornos Cognitivos. Instituto de Investigación Sanitaria del Hospital 12 de Octubre (Imas12). Avenida de Córdoba, s/n. 28041. Madrid. España.\nFecha de ingreso: 21/10/2021\nEpisodio: 24822158\nCentro de salud: Centro de Salud Málaga Oeste\nMatrícula del coche: 5267NPH\nModelo: Mercedes-Benz C-Class\nVIN: VSL4HQZ1ZGI361218\n\nInforme clínico del paciente:\nPaciente de ascendencia australiana de 7 años de edad, acompañado de su tía. Se registró una sesión de escaneo del iris, el código obtenido para revisarla es KJ369UZN325.","Datos del paciente.\nNombre: Victoria Plaza Roca\nDNI: 75398682J\nFecha de nacimiento: 27/09/2014\nGenero: F\nDomicilio: Calle de Arganzuela 18\nCiudad: Gijon, La Rioja, La Rioja\nCodigo postal: 26167\nEmail: victoria_roca@ciberer.es\nTelefono fijo: +34 941 29 92 15\nTelefono movil: +34 641 97 74 21\nNHC: 1980495\nNASS: 620412206059\nCondicion de riesgo: Operador de Maquinaria Pesada\n\nDatos 

### DRX

In [134]:
# ## Expresión regular para DRX

# # Distancia máxima tolerada entre el token trigger y el último que puede ser marcado como esa entidad.
# dist_token = 15

# # originales-------------------------------------------
# #titulos = '((?<=\.| )|^)dr(?=(a|\.| ))|doctor(a?)|((?<=\.| )|^)licenciad[a|o]|((?<=\.| )|^)lic(\.| ?)'
# #-------------------------------------------------------

# regexp_drx = re.compile(r'((?<=\.| )|^)dr(?=(a|\.| |$))|doctor(a?)|((?<=\.| )|^)licenciad[a|o]|((?<=\.| )|^)lic(\.| ?)',
#                         re.IGNORECASE)

# regexp_drx_mayusculas = re.compile(r'^[A-Z]{1}[a-z]+')

# def deidentificar_drx(texto_a_deidentificar):
#     # Genero la lista a partir del texto_preprocesado de esa fila
#     lista_texto_a_deidentificar = re.split('([\.,:\? -])',texto_a_deidentificar)
    
#     ## Iteración entre tokens de la lista del texto a deidentificar
#     drx_flag = False
#     for j,token in enumerate(lista_texto_a_deidentificar):
#         # A partir del segundo token
#         if j > 0:
#             # Si el token actual es precedido por dr/doc,etc
#             if regexp_drx.search(lista_texto_a_deidentificar[j-1]):
#                 drx_flag=True
#                 pos_token = j
#                 # Si el token actual comienza con mayúscula
# #                 if (lista_texto_a_deidentificar[j].lower() in lista_nombres) or regexp_drx_mayusculas.search(lista_texto_a_deidentificar[j]):
# #                 # Reemplazar el token actual por el patrón correspondiente
# #                     if output_tipo==1:
# #                         lista_texto_a_deidentificar[j] = '<'+ lista_texto_a_deidentificar[j] +'>'
# #                     if output_tipo==2:
# #                         lista_texto_a_deidentificar[j] = 'XXXX'
# #                     if output_tipo==3:
# #                         lista_texto_a_deidentificar[j] = '<DRX>'
#             # Si el token actual es un nombre o apellido
# #             if drx == True and (lista_texto_a_deidentificar[j].lower() in lista_nombres or regexp_drx_mayusculas.search(lista_texto_a_deidentificar[j])) and j-pos_token<dist_token:
#             if drx_flag == True and (lista_texto_a_deidentificar[j]=='XPERSONAX' and j-pos_token<dist_token):
#                     # Reemplazar el token actual por el patrón correspondiente
#                     if output_tipo==1:
#                         lista_texto_a_deidentificar[j] = '<'+ lista_texto_a_deidentificar[j] +'>'
#                     if output_tipo==2:
#                         lista_texto_a_deidentificar[j] = 'XXXX'
#                     if output_tipo==3:
#                         lista_texto_a_deidentificar[j] = 'XDRXX'    
# #             else:
# #                 drx_flag == False
    
#     # Generar el texto deidentificado concatenando todos los tokens ya revisados (y modificados si corrrespondiera)
#     return ''.join(lista_texto_a_deidentificar)
    
# textos_a_deidentificar['texto_anonimizado'] = textos_a_deidentificar['texto_anonimizado'].apply(deidentificar_drx)
# #     Aclarar por qué busco en listas (minus y mayus())

### EPOF

Enfermedades poco frecuentes

In [135]:
epof = pd.read_csv("./datasets/epof.csv", names=['enfermedad'], header=None)
lista_epof = epof['enfermedad'].to_list()
lista_epof[:10]

['Abetalipoproteinemia',
 'Abscesos asépticos sensibles a corticosteroides',
 'Acalasia - microcefalia',
 'Acalasia idiopática',
 'Acalvaria',
 'Acatalasemia',
 'Accidente cerebrovascular isquémico pediátrico',
 'Aceruloplasminemia',
 'Acidemia isovalérica',
 'Acidemia malónica y metilmalónica combinada']

In [136]:
epof_names = set([
    remove_accents(name).strip()
    for name in lista_epof if len(name) > 1 and not re.search(ILLEGAL_CHAR_REGEX, name)
])

regexp_epof = '|'.join(epof_names)

epof_anonymizer = RegexAnnonimizer(
    content_regex=regexp_epof,
    category='XEPoFX',
    re_flags=re.IGNORECASE,
    output_type=OUTPUT_TYPE,
)

print(epof_anonymizer.anonymize("Paciente presenta acalvaria aguda."))
print(epof_anonymizer.anonymize("Paciente presenta Acidemia malonica y metilmalonica combinada."))

Paciente presenta XEPoFX aguda.
Paciente presenta XEPoFX.


In [137]:
documents['anonymized'] = documents['anonymized'].apply(epof_anonymizer.anonymize)

In [138]:
documents[documents['anonymized'].str.contains('XEPoFX', case=False)]

,original,preprocessed,anonymized
42,"Datos del paciente.\nNombre: Vanessa Fernández Hidalgo\nDNI: 81323063P\nFecha de nacimiento: 06/10/1955\nGénero: M\nDomicilio: Calle de Amor de Dios 75, 4N\nCiudad: Ciudad autónoma de Ceuta, Ceuta, \nCódigo postal: 52956\nEmail: vanessahidalgo536@gmail.com\nTeléfono fijo: +34 956 49 68 35\nTeléfono móvil: +34 756 27 78 89\nNHC: 0359081\nNASS: 066639713897\nCondición de riesgo: Conductor de Camión\n\nDatos asistenciales.\nMédico: Dra. Noemí Santos Valero. NC 493742703. Investigadora Principal en Parkinson. Instituto de Investigación Biomédica en Red de Enfermedades Neurodegenerativas (CIBERNED). Avenida Monforte de Lemos 3-5. 28029. Madrid. España.\nFecha de ingreso: 12/03/1984\nHospital: Hospital Clínic Universitario de Valencia\nMatrícula del coche: 1694JZP\nModelo: Ram 1500\nVIN: VSCYFG5QP0R652821\n\nInforme clínico del paciente:\nPaciente vegetariano de 28 años de edad.","Datos del paciente.\nNombre: Vanessa Fernandez Hidalgo\nDNI: 81323063P\nFecha de nacimiento: 06/10/1955\nGenero: M\nDomicilio: Calle de Amor de Dios 75, 4N\nCiudad: Ciudad autonoma de Ceuta, Ceuta, \nCodigo postal: 52956\nEmail: vanessahidalgo536@gmail.com\nTelefono fijo: +34 956 49 68 35\nTelefono movil: +34 756 27 78 89\nNHC: 0359081\nNASS: 066639713897\nCondicion de riesgo: Conductor de Camion\n\nDatos asistenciales.\nMedico: Dra. Noemi Santos Valero. NC 493742703. Investigadora Principal en Parkinson. Instituto de Investigacion Biomedica en Red de Enfermedades Neurodegenerativas (CIBERNED). Avenida Monforte de Lemos 3-5. 28029. Madrid. España.\nFecha de ingreso: 12/03/1984\nHospital: Hospital Clinic Universitario de Valencia\nMatricula del coche: 1694JZP\nModelo: Ram 1500\nVIN: VSCYFG5QP0R652821\n\nInforme clinico del paciente:\nPaciente vegetariano de 28 años de edad.","Datos del paciente.\nNombre: XPERSONAX\nDNI: 81323063P\nFecha de nacimiento: XFECHAX\nGenero: M\nDomicilio: Calle XDIRECCIONX Amor de Dios 75, 4N\nXPERSONAX: XPERSONAX autoXEPoFX de Ceuta, Ceuta, \nCodigo postal: 52956\nEmail: XCORREO_ELECTRONICOX\nTelefono fijo: +34 9XFECHAX 35\nTelefono movil: +34 7XFECHAX 89\nNHC: 0359081\nXPERSONAX: 066639713897\nCondicion de riesgo: Conductor de Camion\n\nDatos asistenciales.\nMedico: Dra. XPERSONAX. NC 493742703. Investigadora Principal en Parkinson. Instituto de Investigacion Biomedica en Red de Enfermedades Neurodegenerativas (CIBERNED). Avenida XDIRECCIONX de XPERSONAX 3-5. 28029. XPERSONAX. XPAISX.\nFecha de ingreso: XFECHAX\nHospital: Hospital Clinic Universitario de XPERSONAX\nMatricula del XPERSONAX: 1694JZP\nModelo: Ram 1500\nVIN: VSCYFG5QP0R652821\n\nInforme clinico del paciente:\nPaciente vegetariano de 28 años de edad."
62,"Datos del paciente.\nNombre: Miriam Contreras Escudero\nDNI: 80827695Z\nFecha de nacimiento: 09/05/2004\nGénero: M\nDomicilio: Calle del Duque de Alba 58\nCiudad: Ciudad autónoma de Ceuta, Ceuta, \nCódigo postal: 52023\nEmail: LS0mn@uhu.es\nTeléfono fijo: +34 956 46 20 02\nTeléfono móvil: +34 656 90 48 64\nNHC: 5163263\nNASS: 781658362040\n\nDatos asistenciales.\nMédico: Dra. María Antonia López Robles. NC 049471038. Psiquiatra Especialista en Trastornos de Ansiedad. Instituto de Investigación Biomédica en Red de Salud Mental (CIBERSAM). Avenida Monforte de Lemos 3-5. 28029. Madrid. España.\nFecha de ingreso: 28/02/2024\nHospital: Hospital General Universitario Morales Meseguer\nMatrícula del coche: 4463EXT\nModelo: Ford F-150\nVIN: VSLHNKB2R9G964314\n\nInforme clínico del paciente:\nPaciente fumador de 19 años de edad, acompañado de su compañero de trabajo.","Datos del paciente.\nNombre: Miriam Contreras Escudero\nDNI: 80827695Z\nFecha de nacimiento: 09/05/2004\nGenero: M\nDomicilio: Calle del Duque de Alba 58\nCiudad: Ciudad autonoma de Ceuta, Ceuta, \nCodigo postal: 52023\nEmail: LS0mn@uhu.es\nTelefono fijo: +34 956 46 20 02\nTelefono movil: +34 656 90 48 64\nNHC: 5163263\nNASS: 781658362040\n\nDatos asistenciales.\nMedico: Dra. Maria Antonia Lopez Robles. NC 049471038. Psiqu

## 5. Devuelve los datos procesados (output)

### Transforma entidades

## 6. Guarda salida

In [139]:
documents.shape
output_filename =  '/home/usuario/Documentos/TrabajoEspecial/Modelos/REGEX/b) SPG/out.csv'
print('Saving to:', output_filename)
documents.to_csv(output_filename, sep=',', header=True, index=False)

Saving to: /home/usuario/Documentos/TrabajoEspecial/Modelos/REGEX/b) SPG/out.csv
